Título del Trabajo: Diseño e implementación de un mecanismo ETL que permita la integración de fuentes públicas del gobierno local para estudios de siniestralidad vial en Bogotá

Universidad: Universidad Central

Facultad: Facultad de Ingeniería y Ciencias Básicas

Programa Académico: Maestría en Analítica de Datos

Directores:
- Alejandra Baena, PhD
-Juan Camilo Ramírez, PhD

Autores:
- Natalia Garzón Arias
- Luz Adriana Jiménez Sabrica


Fecha de Presentación: 05-febrero-2024




# Tabla de Contenido

1. [Introducción](#introduccion)
2. [Extracción](#extraccion)
3. [Transformación](#transformacion)
4. [Carga](#carga)
5. [Interfaz de Monitoreo](#Interfaz)

## 1. Introducción


El propósito de las siguientes líneas de código es crear un mecanismo de Extracción, Transformación y Carga (ETL, por sus siglas en inglés) para la generación de conocimiento a partir de fuentes gubernamentales públicas, que hoy exponen los datos de siniestros viales de manera aislada dificultando ver las relaciones o influencias entre las variables disponibles.

## 2. Extracción <a id="extraccion"></a>

In [1]:
import time
import psutil
import os
from bs4 import BeautifulSoup
from urllib.request import urlopen

In [2]:
# Inicia el temporizador para la fase de Extracción
extraction_start_time = time.time()

In [3]:
# Instalamos las dependencias para instalar Google Chrome
!sudo apt-get update
!sudo apt-get install -y libu2f-udev_1.1.4-1_all.deb
!pip install selenium

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,352 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,848 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports/universe amd64 Packages [33.3 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-backports/m

In [4]:
# Instalamos Google-Chrome
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb
!apt-get install -f
!rm -rf google-chrome-stable_current_amd64.deb

--2024-03-13 23:43:55--  https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
Resolving dl.google.com (dl.google.com)... 142.250.141.190, 142.250.141.136, 142.250.141.93, ...
Connecting to dl.google.com (dl.google.com)|142.250.141.190|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 106053828 (101M) [application/x-debian-package]
Saving to: ‘google-chrome-stable_current_amd64.deb’

google-chrome-stabl 100%[===================>] 101.14M   192MB/s    in 0.5s    

2024-03-13 23:43:56 (192 MB/s) - ‘google-chrome-stable_current_amd64.deb’ saved [106053828/106053828]

Selecting previously unselected package google-chrome-stable.
(Reading database ... 121752 files and directories currently installed.)
Preparing to unpack google-chrome-stable_current_amd64.deb ...
Unpacking google-chrome-stable (122.0.6261.128-1) ...
dpkg: dependency problems prevent configuration of google-chrome-stable:
 google-chrome-stable depends on libu2f-udev; however:
  Pac

In [5]:
# Identificamos la versión estable disponible para descarga del ChromeWebDriver
url = "https://googlechromelabs.github.io/chrome-for-testing/"
page = urlopen(url)
html = page.read().decode("utf-8")
soup = BeautifulSoup(html, "html.parser")
code=soup.find("code")

In [27]:
# Descargamos el WebDriver para Chrome
ver_Google = str(code)
ver_Google = ver_Google[len ("<code>"):(len(ver_Google)-len ("</code>"))]
url_WebDriver = 'https://storage.googleapis.com/chrome-for-testing-public/'+ver_Google+'/linux64/chromedriver-linux64.zip'
!wget $url_WebDriver
!unzip chromedriver-linux64
!rm -rf chromedriver-linux64.zip
!rm -rf LICENSE.chromedriver

--2024-03-14 01:41:14--  https://storage.googleapis.com/chrome-for-testing-public/122.0.6261.128/linux64/chromedriver-linux64.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.101.207, 142.250.141.207, 142.251.2.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.101.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8598044 (8.2M) [application/zip]
Saving to: ‘chromedriver-linux64.zip’

chromedriver-linux6 100%[===================>]   8.20M  --.-KB/s    in 0.1s    

2024-03-14 01:41:14 (76.3 MB/s) - ‘chromedriver-linux64.zip’ saved [8598044/8598044]

Archive:  chromedriver-linux64.zip
replace chromedriver-linux64/LICENSE.chromedriver? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [29]:
# importamos el modulo webdriver de Selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
import os


# Opciones de chrome
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')


# Ruta del controlador de Chrome
chrome_driver_path = '/content/chromedriver-linux64/chromedriver'

# Crear un objeto Service para especificar la ruta del controlador
service = Service(chrome_driver_path)

# Crear una instancia del navegador Chrome
driver = webdriver.Chrome(service=service, options=chrome_options)

In [8]:
# Definir función para acceder a shadow_root
def expand_shadow_element(element):
    shadow_root = driver.execute_script('return arguments[0].shadowRoot', element)
    return shadow_root

### 2.1 Actor Vial

In [32]:
# Navegar en la página web
driver.get('https://datos.movilidadbogota.gov.co/datasets/movilidadbogota::actor-vial/about')
time.sleep(7)

download = WebDriverWait(driver, 20).until(
          EC.element_to_be_clickable((By.XPATH,
                                      '/html[1]/body[1]/div[7]/div[2]/div[1]/div[1]/div[3]/div[1]/div[1]/div[4]/div[2]/button[1]')))
download.click()

root1 = driver.find_element(By.CSS_SELECTOR, "arcgis-download-list")
shadow_root1 = expand_shadow_element(root1)
time.sleep(5)

root2 = shadow_root1.find_element(By.CSS_SELECTOR, "arcgis-download-list-card.list-card")
shadow_root2 = expand_shadow_element(root2)
time.sleep(5)

root3 = shadow_root2.find_element(By.CSS_SELECTOR, "arcgis-download-list-card-ui")
shadow_root3 = expand_shadow_element(root3)
time.sleep(5)

root4 = shadow_root2.find_element(By.CSS_SELECTOR, 'arcgis-download-features-button')
shadow_root4 = expand_shadow_element(root4)
time.sleep(5)

root5 = shadow_root4.find_element(By.CSS_SELECTOR, 'calcite-button')
shadow_root5 = expand_shadow_element(root5)
time.sleep(5)

csv_download = shadow_root5.find_element(By.CSS_SELECTOR, ".button-padding--shrunk.content--slotted.icon-end-empty")
driver.execute_script("arguments[0].scrollIntoView();", csv_download)
driver.execute_script("arguments[0].click();", csv_download)
time.sleep(20)

In [ ]:
# Obtener lista de archivos en el directorio
archivos = os.listdir('/content')

# Variable para verificar si el archivo ha aparecido
archivo_aparecido = False

# Tiempo máximo de espera en segundos
tiempo_maximo_espera = 300  # Por ejemplo, 5 minutos

# Tiempo de espera entre verificaciones en segundos
intervalo_verificacion = 10

tiempo_transcurrido = 0

while not archivo_aparecido and tiempo_transcurrido < tiempo_maximo_espera:
    if any(archivo.endswith('.csv') and archivo.startswith('AccidentalidadAnalisis') for archivo in archivos):
        archivo_aparecido = True
    else:
        tiempo_transcurrido += intervalo_verificacion
        time.sleep(intervalo_verificacion)

if archivo_aparecido:
  # Iterar sobre los archivos
  for archivo in archivos:
      # Verificar si el nombre del archivo comienza con 'AccidentalidadAnalisis' y termina con '.csv'
      if archivo.startswith('AccidentalidadAnalisis') and archivo.endswith('.csv'):
          # Ruta completa del archivo original
          OriginalActorVial = '/content/' + archivo

          # Nuevo nombre del archivo (eliminando los números al final)
          nuevo_nombre = 'ActorVial.csv'

          # Nueva ruta con el nombre modificado
          NuevaActorVial = '/content/' + nuevo_nombre

          # Cambiar el nombre del archivo
          os.rename(OriginalActorVial, NuevaActorVial)
else:
    print("El archivo no ha aparecido en la carpeta dentro del tiempo de espera.")

### 2.2 Via

In [34]:
# Navego en la página web
driver.get('https://datos.movilidadbogota.gov.co/datasets/movilidadbogota::via/about')
time.sleep(7)

download = WebDriverWait(driver, 20).until(
          EC.element_to_be_clickable((By.XPATH,
                                      '/html[1]/body[1]/div[7]/div[2]/div[1]/div[1]/div[3]/div[1]/div[1]/div[4]/div[2]/button[1]')))
download.click()

root1 = driver.find_element(By.CSS_SELECTOR, "arcgis-download-list")
shadow_root1 = expand_shadow_element(root1)
time.sleep(5)

root2 = shadow_root1.find_element(By.CSS_SELECTOR, "arcgis-download-list-card.list-card")
shadow_root2 = expand_shadow_element(root2)
time.sleep(5)

root3 = shadow_root2.find_element(By.CSS_SELECTOR, "arcgis-download-list-card-ui")
shadow_root3 = expand_shadow_element(root3)
time.sleep(5)

root4 = shadow_root2.find_element(By.CSS_SELECTOR, 'arcgis-download-features-button')
shadow_root4 = expand_shadow_element(root4)
time.sleep(5)

root5 = shadow_root4.find_element(By.CSS_SELECTOR, 'calcite-button')
shadow_root5 = expand_shadow_element(root5)
time.sleep(5)

csv_download = shadow_root5.find_element(By.CSS_SELECTOR, ".button-padding--shrunk.content--slotted.icon-end-empty")
driver.execute_script("arguments[0].scrollIntoView();", csv_download)
driver.execute_script("arguments[0].click();", csv_download)
time.sleep(20)

In [35]:
# Obtener lista de archivos en el directorio
archivos = os.listdir('/content')

# Variable para verificar si el archivo ha aparecido
archivo_aparecido = False

# Tiempo máximo de espera en segundos
tiempo_maximo_espera = 300  # Por ejemplo, 5 minutos

# Tiempo de espera entre verificaciones en segundos
intervalo_verificacion = 10

tiempo_transcurrido = 0

while not archivo_aparecido and tiempo_transcurrido < tiempo_maximo_espera:
    if any(archivo.endswith('.csv') and archivo.startswith('AccidentalidadAnalisis') for archivo in archivos):
        archivo_aparecido = True
    else:
        tiempo_transcurrido += intervalo_verificacion
        time.sleep(intervalo_verificacion)

if archivo_aparecido:
  # Iterar sobre los archivos
  for archivo in archivos:
      # Verificar si el nombre del archivo comienza con 'AccidentalidadAnalisis' y termina con '.csv'
      if archivo.startswith('AccidentalidadAnalisis') and archivo.endswith('.csv'):
          # Ruta completa del archivo original
          OriginalVia = '/content/' + archivo

          # Nuevo nombre del archivo (eliminando los números al final)
          nuevo_nombre = 'Via.csv'

          # Nueva ruta con el nombre modificado
          NuevaVia = '/content/' + nuevo_nombre

          # Cambiar el nombre del archivo
          os.rename(OriginalVia, NuevaVia)
else:
    print("El archivo no ha aparecido en la carpeta dentro del tiempo de espera.")

### 2.3 Accidente

In [36]:
# Navego en la página web
driver.get('https://datos.movilidadbogota.gov.co/datasets/movilidadbogota::accidente-1/about')
time.sleep(7)

download = WebDriverWait(driver, 20).until(
          EC.element_to_be_clickable((By.XPATH,
                                      '/html[1]/body[1]/div[7]/div[2]/div[1]/div[1]/div[3]/div[1]/div[1]/div[4]/div[2]/button[1]')))
download.click()

root1 = driver.find_element(By.CSS_SELECTOR, "arcgis-download-list")
shadow_root1 = expand_shadow_element(root1)
time.sleep(5)

root2 = shadow_root1.find_element(By.CSS_SELECTOR, "arcgis-download-list-card.list-card")
shadow_root2 = expand_shadow_element(root2)
time.sleep(5)

root3 = shadow_root2.find_element(By.CSS_SELECTOR, "arcgis-download-list-card-ui")
shadow_root3 = expand_shadow_element(root3)
time.sleep(5)

root4 = shadow_root2.find_element(By.CSS_SELECTOR, 'arcgis-download-features-button')
shadow_root4 = expand_shadow_element(root4)
time.sleep(5)

root5 = shadow_root4.find_element(By.CSS_SELECTOR, 'calcite-button')
shadow_root5 = expand_shadow_element(root5)
time.sleep(5)

csv_download = shadow_root5.find_element(By.CSS_SELECTOR, ".button-padding--shrunk.content--slotted.icon-end-empty")
driver.execute_script("arguments[0].scrollIntoView();", csv_download)
driver.execute_script("arguments[0].click();", csv_download)
time.sleep(20)

In [37]:
# Obtener lista de archivos en el directorio
archivos = os.listdir('/content')

# Variable para verificar si el archivo ha aparecido
archivo_aparecido = False

# Tiempo máximo de espera en segundos
tiempo_maximo_espera = 300  # Por ejemplo, 5 minutos

# Tiempo de espera entre verificaciones en segundos
intervalo_verificacion = 10

tiempo_transcurrido = 0

while not archivo_aparecido and tiempo_transcurrido < tiempo_maximo_espera:
    if any(archivo.endswith('.csv') and archivo.startswith('AccidentalidadAnalisis') for archivo in archivos):
        archivo_aparecido = True
    else:
        tiempo_transcurrido += intervalo_verificacion
        time.sleep(intervalo_verificacion)

if archivo_aparecido:
  # Iterar sobre los archivos
  for archivo in archivos:
      # Verificar si el nombre del archivo comienza con 'AccidentalidadAnalisis' y termina con '.csv'
      if archivo.startswith('AccidentalidadAnalisis') and archivo.endswith('.csv'):
          # Ruta completa del archivo original
          OriginalAccidente = '/content/' + archivo

          # Nuevo nombre del archivo (eliminando los números al final)
          nuevo_nombre = 'Accidente.csv'

          # Nueva ruta con el nombre modificado
          NuevaAccidente = '/content/' + nuevo_nombre

          # Cambiar el nombre del archivo
          os.rename(OriginalAccidente, NuevaAccidente)
else:
    print("El archivo no ha aparecido en la carpeta dentro del tiempo de espera.")

### 2.4 Vehiculo

In [38]:
# Navego en la página web
driver.get('https://datos.movilidadbogota.gov.co/datasets/movilidadbogota::vehiculo-1/about')
time.sleep(7)

download = WebDriverWait(driver, 20).until(
          EC.element_to_be_clickable((By.XPATH,
                                      '/html[1]/body[1]/div[7]/div[2]/div[1]/div[1]/div[3]/div[1]/div[1]/div[4]/div[2]/button[1]')))
download.click()

root1 = driver.find_element(By.CSS_SELECTOR, "arcgis-download-list")
shadow_root1 = expand_shadow_element(root1)
time.sleep(5)

root2 = shadow_root1.find_element(By.CSS_SELECTOR, "arcgis-download-list-card.list-card")
shadow_root2 = expand_shadow_element(root2)
time.sleep(5)

root3 = shadow_root2.find_element(By.CSS_SELECTOR, "arcgis-download-list-card-ui")
shadow_root3 = expand_shadow_element(root3)
time.sleep(5)

root4 = shadow_root2.find_element(By.CSS_SELECTOR, 'arcgis-download-features-button')
shadow_root4 = expand_shadow_element(root4)
time.sleep(5)

root5 = shadow_root4.find_element(By.CSS_SELECTOR, 'calcite-button')
shadow_root5 = expand_shadow_element(root5)
time.sleep(5)

csv_download = shadow_root5.find_element(By.CSS_SELECTOR, ".button-padding--shrunk.content--slotted.icon-end-empty")
driver.execute_script("arguments[0].scrollIntoView();", csv_download)
driver.execute_script("arguments[0].click();", csv_download)
time.sleep(20)

In [39]:
# Obtener lista de archivos en el directorio
archivos = os.listdir('/content')

# Variable para verificar si el archivo ha aparecido
archivo_aparecido = False

# Tiempo máximo de espera en segundos
tiempo_maximo_espera = 300  # Por ejemplo, 5 minutos

# Tiempo de espera entre verificaciones en segundos
intervalo_verificacion = 10

tiempo_transcurrido = 0

while not archivo_aparecido and tiempo_transcurrido < tiempo_maximo_espera:
    if any(archivo.endswith('.csv') and archivo.startswith('AccidentalidadAnalisis') for archivo in archivos):
        archivo_aparecido = True
    else:
        tiempo_transcurrido += intervalo_verificacion
        time.sleep(intervalo_verificacion)

if archivo_aparecido:
  # Iterar sobre los archivos
  for archivo in archivos:
      # Verificar si el nombre del archivo comienza con 'AccidentalidadAnalisis' y termina con '.csv'
      if archivo.startswith('AccidentalidadAnalisis') and archivo.endswith('.csv'):
          # Ruta completa del archivo original
          OriginalVehiculo = '/content/' + archivo

          # Nuevo nombre del archivo (eliminando los números al final)
          nuevo_nombre = 'Vehiculo.csv'

          # Nueva ruta con el nombre modificado
          NuevaVehiculo = '/content/' + nuevo_nombre

          # Cambiar el nombre del archivo
          os.rename(OriginalVehiculo, NuevaVehiculo)
else:
    print("El archivo no ha aparecido en la carpeta dentro del tiempo de espera.")

### 2.5 Causa

In [40]:
# Navego en la página web
driver.get('https://datos.movilidadbogota.gov.co/datasets/movilidadbogota::causa/about')
time.sleep(7)

download = WebDriverWait(driver, 20).until(
          EC.element_to_be_clickable((By.XPATH,
                                      '/html[1]/body[1]/div[7]/div[2]/div[1]/div[1]/div[3]/div[1]/div[1]/div[4]/div[2]/button[1]')))
download.click()

root1 = driver.find_element(By.CSS_SELECTOR, "arcgis-download-list")
shadow_root1 = expand_shadow_element(root1)
time.sleep(5)

root2 = shadow_root1.find_element(By.CSS_SELECTOR, "arcgis-download-list-card.list-card")
shadow_root2 = expand_shadow_element(root2)
time.sleep(5)

root3 = shadow_root2.find_element(By.CSS_SELECTOR, "arcgis-download-list-card-ui")
shadow_root3 = expand_shadow_element(root3)
time.sleep(5)

root4 = shadow_root2.find_element(By.CSS_SELECTOR, 'arcgis-download-features-button')
shadow_root4 = expand_shadow_element(root4)
time.sleep(5)

root5 = shadow_root4.find_element(By.CSS_SELECTOR, 'calcite-button')
shadow_root5 = expand_shadow_element(root5)
time.sleep(5)

csv_download = shadow_root5.find_element(By.CSS_SELECTOR, ".button-padding--shrunk.content--slotted.icon-end-empty")
driver.execute_script("arguments[0].scrollIntoView();", csv_download)
driver.execute_script("arguments[0].click();", csv_download)
time.sleep(20)

In [42]:
# Obtener lista de archivos en el directorio
archivos = os.listdir('/content')

# Variable para verificar si el archivo ha aparecido
archivo_aparecido = False

# Tiempo máximo de espera en segundos
tiempo_maximo_espera = 300  # Por ejemplo, 5 minutos

# Tiempo de espera entre verificaciones en segundos
intervalo_verificacion = 10

tiempo_transcurrido = 0

while not archivo_aparecido and tiempo_transcurrido < tiempo_maximo_espera:
    if any(archivo.endswith('.csv') and archivo.startswith('AccidentalidadAnalisis') for archivo in archivos):
        archivo_aparecido = True
    else:
        tiempo_transcurrido += intervalo_verificacion
        time.sleep(intervalo_verificacion)

if archivo_aparecido:
  # Iterar sobre los archivos
  for archivo in archivos:
      # Verificar si el nombre del archivo comienza con 'AccidentalidadAnalisis' y termina con '.csv'
      if archivo.startswith('AccidentalidadAnalisis') and archivo.endswith('.csv'):
          # Ruta completa del archivo original
          OriginalCausa = '/content/' + archivo

          # Nuevo nombre del archivo (eliminando los números al final)
          nuevo_nombre = 'Causa.csv'

          # Nueva ruta con el nombre modificado
          NuevaCausa = '/content/' + nuevo_nombre

          # Cambiar el nombre del archivo
          os.rename(OriginalCausa, NuevaCausa)
else:
    print("El archivo no ha aparecido en la carpeta dentro del tiempo de espera.")

In [ ]:
# Detiene el temporizador de extracción
extraction_end_time = time.time()

In [ ]:
# Calcula el tiempo de extracción
extraction_time = extraction_end_time - extraction_start_time
print(f'Tiempo de extracción: {extraction_time:.2f} segundos')

Tiempo de extracción: 198.87 segundos


## 3. Transformación <a id="transformacion"></a>

In [ ]:
# Inicia el temporizador de transformación
transformation_start_time = time.time()

In [ ]:
import pandas as pd
import numpy as np
import datetime

### 3.1 Creación tabla Geometrica A

In [ ]:
# 1. Crear la tabla de Geometrica_A con los campos necesarios
Geometrica_A = pd.read_csv('/content/Via.csv')
Geometrica_A = Geometrica_A[['GEOMETRICA_A']]

<ipython-input-24-0d072ac0676e>:2: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  Geometrica_A = pd.read_csv('/content/Via.csv')


In [ ]:
# 2. Revisión de la Estructura del Dataset
print('El DataFrame está compuesto por {:,}'.format(Geometrica_A.shape[0]),'filas y',Geometrica_A.shape[1],'columnas')

El DataFrame está compuesto por 483,952 filas y 1 columnas


In [ ]:
# 3. Eliminamos los duplicados
Geometrica_A = Geometrica_A.drop_duplicates()

In [ ]:
# 4. Eliminamos los valores vacíos
Geometrica_A = Geometrica_A.dropna()

In [ ]:
# 5. Reseteamos el índice y confimamos la Estructura del Dataset
Geometrica_A = Geometrica_A.reset_index(drop=True)
print('El DataFrame está compuesto por {:,}'.format(Geometrica_A.shape[0]),'filas y',Geometrica_A.shape[1],'columnas')

El DataFrame está compuesto por 2 filas y 1 columnas


In [ ]:
#6. Creamos codigo de geometrica A
Geometrica_A['COD_GEOMETRICA_A']=Geometrica_A.index+1

In [ ]:
print(Geometrica_A.head())

  GEOMETRICA_A  COD_GEOMETRICA_A
0        RECTA                 1
1        CURVA                 2


In [ ]:
Geometrica_A["COD_GEOMETRICA_A"] = pd.to_numeric(Geometrica_A["COD_GEOMETRICA_A"] , downcast="integer", errors="coerce")
Geometrica_A["COD_GEOMETRICA_A"] = Geometrica_A["COD_GEOMETRICA_A"].astype(int)

In [ ]:
print('Las variables almacenan la información en el siguiente formato:', Geometrica_A.dtypes)

Las variables almacenan la información en el siguiente formato: GEOMETRICA_A        object
COD_GEOMETRICA_A     int64
dtype: object


### 3.2 Creación tabla Geometrica B

In [ ]:
# 1. Crear la tabla de Geometrica_B con los campos necesarios
Geometrica_B = pd.read_csv('/content/Via.csv')
Geometrica_B = Geometrica_B[['GEOMETRICA_B']]

<ipython-input-33-74611000e084>:2: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  Geometrica_B = pd.read_csv('/content/Via.csv')


In [ ]:
# 2. Revisión de la Estructura del Dataset
print('El DataFrame está compuesto por {:,}'.format(Geometrica_B.shape[0]),'filas y',Geometrica_B.shape[1],'columnas')

El DataFrame está compuesto por 483,952 filas y 1 columnas


In [ ]:
# 3. Eliminamos los duplicados
Geometrica_B = Geometrica_B.drop_duplicates()

In [ ]:
# 4. Eliminamos los valores vacíos
Geometrica_B = Geometrica_B.dropna()

In [ ]:
# 5. Reseteamos el ínidce y confimamos  de la Estructura del Dataset
Geometrica_B = Geometrica_B.reset_index(drop=True)
print('El DataFrame está compuesto por {:,}'.format(Geometrica_B.shape[0]),'filas y',Geometrica_B.shape[1],'columnas')

El DataFrame está compuesto por 2 filas y 1 columnas


In [ ]:
#6. Creamos codigo de geometrica B
Geometrica_B['COD_GEOMETRICA_B']=Geometrica_B.index+1

In [ ]:
print(Geometrica_B.head())

  GEOMETRICA_B  COD_GEOMETRICA_B
0        PLANO                 1
1    PENDIENTE                 2


In [ ]:
Geometrica_B["COD_GEOMETRICA_B"] = pd.to_numeric(Geometrica_B["COD_GEOMETRICA_B"] , downcast="integer", errors="coerce")
Geometrica_B["COD_GEOMETRICA_B"] = Geometrica_B["COD_GEOMETRICA_B"].astype(int)

In [ ]:
print('Las variables almacenan la información en el siguiente formato:', Geometrica_B.dtypes)

Las variables almacenan la información en el siguiente formato: GEOMETRICA_B        object
COD_GEOMETRICA_B     int64
dtype: object


### 3.3 Creación tabla Geometrica C

In [ ]:
# 1. Crear la tabla de Geometrica_C con los campos necesarios
Geometrica_C = pd.read_csv('/content/Via.csv')
Geometrica_C = Geometrica_C[['GEOMETRICA_C']]

<ipython-input-42-43fbe1886a20>:2: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  Geometrica_C = pd.read_csv('/content/Via.csv')


In [ ]:
# 2. Revisión de la Estructura del Dataset
print('El DataFrame está compuesto por {:,}'.format(Geometrica_C.shape[0]),'filas y',Geometrica_C.shape[1],'columnas')

El DataFrame está compuesto por 483,952 filas y 1 columnas


In [ ]:
# 3. Eliminamos los duplicados
Geometrica_C = Geometrica_C.drop_duplicates()

In [ ]:
# 4. Eliminamos los valores vacíos
Geometrica_C = Geometrica_C.dropna()

In [ ]:
# 5. Reseteamos el ínidce y confimamos  de la Estructura del Dataset
Geometrica_C = Geometrica_C.reset_index(drop=True)
print('El DataFrame está compuesto por {:,}'.format(Geometrica_C.shape[0]),'filas y',Geometrica_C.shape[1],'columnas')

El DataFrame está compuesto por 3 filas y 1 columnas


In [ ]:
#6. Creamos codigo de geometrica c
Geometrica_C['COD_GEOMETRICA_C']=Geometrica_C.index+1

In [ ]:
print(Geometrica_C.head())

    GEOMETRICA_C  COD_GEOMETRICA_C
0      CON ANDEN                 1
1     CON BERMAS                 2
2  BAHIA DE EST.                 3


In [ ]:
Geometrica_C["COD_GEOMETRICA_C"] = pd.to_numeric(Geometrica_C["COD_GEOMETRICA_C"] , downcast="integer", errors="coerce")
Geometrica_C["COD_GEOMETRICA_C"] = Geometrica_C["COD_GEOMETRICA_C"].astype(int)

In [ ]:
print('Las variables almacenan la información en el siguiente formato:', Geometrica_C.dtypes)

Las variables almacenan la información en el siguiente formato: GEOMETRICA_C        object
COD_GEOMETRICA_C     int64
dtype: object


### 3.4 Creación tabla Utilizacion

In [ ]:
# 1. Crear la tabla de Utilizacion con los campos necesarios
Utilizacion = pd.read_csv('/content/Via.csv')
Utilizacion = Utilizacion[['UTILIZACION']]

<ipython-input-51-d878a33e2166>:2: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  Utilizacion = pd.read_csv('/content/Via.csv')


In [ ]:
# 2. Revisión de la Estructura del Dataset
print('El DataFrame está compuesto por {:,}'.format(Utilizacion.shape[0]),'filas y',Utilizacion.shape[1],'columnas')

El DataFrame está compuesto por 483,952 filas y 1 columnas


In [ ]:
# 3. Eliminamos los duplicados
Utilizacion = Utilizacion.drop_duplicates()

In [ ]:
# 4. Eliminamos los valores vacíos
Utilizacion = Utilizacion.dropna()

In [ ]:
# 5. Reseteamos el ínidce y confimamos  de la Estructura del Dataset
Utilizacion = Utilizacion.reset_index(drop=True)
print('El DataFrame está compuesto por {:,}'.format(Utilizacion.shape[0]),'filas y',Utilizacion.shape[1],'columnas')

El DataFrame está compuesto por 5 filas y 1 columnas


In [ ]:
#6. Creamos codigo de utilizacion
Utilizacion['COD_UTILIZACION']=Utilizacion.index+1

In [ ]:
print(Utilizacion.head())

     UTILIZACION  COD_UTILIZACION
0  DOBLE SENTIDO                1
1     UN SENTIDO                2
2     REVERSIBLE                3
3    CONTRAFLUJO                4
4       CICLOVIA                5


In [ ]:
Utilizacion["COD_UTILIZACION"] = pd.to_numeric(Utilizacion["COD_UTILIZACION"] , downcast="integer", errors="coerce")
Utilizacion["COD_UTILIZACION"] = Utilizacion["COD_UTILIZACION"].astype(int)

In [ ]:
print('Las variables almacenan la información en el siguiente formato:', Utilizacion.dtypes)

Las variables almacenan la información en el siguiente formato: UTILIZACION        object
COD_UTILIZACION     int64
dtype: object


### 3.5 Creación tabla Calzadas

In [ ]:
# 1. Crear la tabla de Calzadas con los campos necesarios
Calzadas = pd.read_csv('/content/Via.csv')
Calzadas = Calzadas[['CALZADAS']]

<ipython-input-60-6fda5a08eb29>:2: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  Calzadas = pd.read_csv('/content/Via.csv')


In [ ]:
# 2. Revisión de la Estructura del Dataset
print('El DataFrame está compuesto por {:,}'.format(Calzadas.shape[0]),'filas y',Calzadas.shape[1],'columnas')

El DataFrame está compuesto por 483,952 filas y 1 columnas


In [ ]:
# 3. Eliminamos los duplicados
Calzadas = Calzadas.drop_duplicates()

In [ ]:
# 4. Eliminamos los valores vacíos
Calzadas = Calzadas.dropna()

In [ ]:
# 5. Reseteamos el ínidce y confimamos  de la Estructura del Dataset
Calzadas = Calzadas.reset_index(drop=True)
print('El DataFrame está compuesto por {:,}'.format(Calzadas.shape[0]),'filas y',Calzadas.shape[1],'columnas')

El DataFrame está compuesto por 5 filas y 1 columnas


In [ ]:
#6. Creamos codigo de calzadas
Calzadas['COD_CALZADAS']=Calzadas.index+1

In [ ]:
print(Calzadas.head())

       CALZADAS  COD_CALZADAS
0           UNA             1
1    TRES O MAS             2
2           DOS             3
3      VARIABLE             4
4  CUATRO O MAS             5


In [ ]:
Calzadas["COD_CALZADAS"] = pd.to_numeric(Calzadas["COD_CALZADAS"] , downcast="integer", errors="coerce")
Calzadas["COD_CALZADAS"] = Calzadas["COD_CALZADAS"].astype(int)

In [ ]:
print('Las variables almacenan la información en el siguiente formato:', Calzadas.dtypes)

Las variables almacenan la información en el siguiente formato: CALZADAS        object
COD_CALZADAS     int64
dtype: object


### 3.6 Creación tabla Carriles

In [ ]:
# 1. Crear la tabla de Carriles con los campos necesarios
Carriles = pd.read_csv('/content/Via.csv')
Carriles = Carriles[['CARRILES']]

<ipython-input-69-34e4608b8eb9>:2: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  Carriles = pd.read_csv('/content/Via.csv')


In [ ]:
# 2. Revisión de la Estructura del Dataset
print('El DataFrame está compuesto por {:,}'.format(Carriles.shape[0]),'filas y',Carriles.shape[1],'columnas')

El DataFrame está compuesto por 483,952 filas y 1 columnas


In [ ]:
# 3. Eliminamos los duplicados
Carriles = Carriles.drop_duplicates()

In [ ]:
# 4. Eliminamos los valores vacíos
Carriles = Carriles.dropna()

In [ ]:
# 5. Reseteamos el ínidce y confimamos  de la Estructura del Dataset
Carriles = Carriles.reset_index(drop=True)
print('El DataFrame está compuesto por {:,}'.format(Carriles.shape[0]),'filas y',Carriles.shape[1],'columnas')

El DataFrame está compuesto por 5 filas y 1 columnas


In [ ]:
#6. Creamos codigo de carriles
Carriles['COD_CARRILES']=Carriles.index+1

In [ ]:
print(Carriles.head())

       CARRILES  COD_CARRILES
0           UNO             1
1           DOS             2
2    TRES O MAS             3
3      VARIABLE             4
4  CUATRO O MAS             5


In [ ]:
Carriles["COD_CARRILES"] = pd.to_numeric(Carriles["COD_CARRILES"] , downcast="integer", errors="coerce")
Carriles["COD_CARRILES"] = Carriles["COD_CARRILES"].astype(int)

In [ ]:
print('Las variables almacenan la información en el siguiente formato:', Carriles.dtypes)

Las variables almacenan la información en el siguiente formato: CARRILES        object
COD_CARRILES     int64
dtype: object


### 3.7 Creación tabla Material

In [ ]:
# 1. Crear la tabla de Material con los campos necesarios
Material = pd.read_csv('/content/Via.csv')
Material = Material[['MATERIAL']]

<ipython-input-78-41b78a2ec93e>:2: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  Material = pd.read_csv('/content/Via.csv')


In [ ]:
# 2. Revisión de la Estructura del Dataset
print('El DataFrame está compuesto por {:,}'.format(Material.shape[0]),'filas y',Material.shape[1],'columnas')

El DataFrame está compuesto por 483,952 filas y 1 columnas


In [ ]:
# 3. Eliminamos los duplicados
Material = Material.drop_duplicates()

In [ ]:
# 4. Eliminamos los valores vacíos
Material = Material.dropna()

In [ ]:
# 5. Reseteamos el ínidce y confimamos  de la Estructura del Dataset
Material = Material.reset_index(drop=True)
print('El DataFrame está compuesto por {:,}'.format(Material.shape[0]),'filas y',Material.shape[1],'columnas')

El DataFrame está compuesto por 7 filas y 1 columnas


In [ ]:
#6. Creamos codigo de material
Material['COD_MATERIAL']=Material.index+1

In [ ]:
print(Material.head(7))

    MATERIAL  COD_MATERIAL
0   CONCRETO             1
1    ASFALTO             2
2   AFIRMADO             3
3    ADOQUIN             4
4     TIERRA             5
5  EMPEDRADO             6
6       OTRO             7


In [ ]:
Material["COD_MATERIAL"] = pd.to_numeric(Material["COD_MATERIAL"] , downcast="integer", errors="coerce")
Material["COD_MATERIAL"] = Material["COD_MATERIAL"].astype(int)

In [ ]:
print('Las variables almacenan la información en el siguiente formato:', Material.dtypes)

Las variables almacenan la información en el siguiente formato: MATERIAL        object
COD_MATERIAL     int64
dtype: object


### 3.8 Creación tabla Estado_Via

In [ ]:
# 1. Crear la tabla de Estado con los campos necesarios
Estado_Via = pd.read_csv('/content/Via.csv')
Estado_Via = Estado_Via[['ESTADO']]

<ipython-input-87-c9dc7ba53665>:2: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  Estado_Via = pd.read_csv('/content/Via.csv')


In [ ]:
# 2. Revisión de la Estructura del Dataset
print('El DataFrame está compuesto por {:,}'.format(Estado_Via.shape[0]),'filas y',Estado_Via.shape[1],'columnas')

El DataFrame está compuesto por 483,952 filas y 1 columnas


In [ ]:
# 3. Eliminamos los duplicados
Estado_Via = Estado_Via.drop_duplicates()

In [ ]:
# 4. Eliminamos los valores vacíos
Estado_Via = Estado_Via.dropna()

In [ ]:
# 5. Reseteamos el ínidce y confimamos  de la Estructura del Dataset
Estado_Via = Estado_Via.reset_index(drop=True)
print('El DataFrame está compuesto por {:,}'.format(Estado_Via.shape[0]),'filas y',Estado_Via.shape[1],'columnas')

El DataFrame está compuesto por 9 filas y 1 columnas


In [ ]:
#6. Creamos codigo de estado
Estado_Via['COD_ESTADO']=Estado_Via.index+1

In [ ]:
print(Estado_Via.head(9))

          ESTADO  COD_ESTADO
0          BUENO           1
1      DERRUMBES           2
2     CON HUECOS           3
3         RIZADA           4
4   HUNDIMIENTOS           5
5  EN REPARACION           6
6       PARCHADA           7
7       FISURADA           8
8       INUNDADA           9


In [ ]:
Estado_Via["COD_ESTADO"] = pd.to_numeric(Estado_Via["COD_ESTADO"] , downcast="integer", errors="coerce")
Estado_Via["COD_ESTADO"] = Estado_Via["COD_ESTADO"].astype(int)

In [ ]:
print('Las variables almacenan la información en el siguiente formato:', Estado_Via.dtypes)

Las variables almacenan la información en el siguiente formato: ESTADO        object
COD_ESTADO     int64
dtype: object


### 3.9 Creación tabla Condiciones

In [ ]:
# 1. Crear la tabla de Condiciones con los campos necesarios
Condiciones = pd.read_csv('/content/Via.csv')
Condiciones = Condiciones[['CONDICIONES']]

<ipython-input-96-1a449f8deb75>:2: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  Condiciones = pd.read_csv('/content/Via.csv')


In [ ]:
# 2. Revisión de la Estructura del Dataset
print('El DataFrame está compuesto por {:,}'.format(Condiciones.shape[0]),'filas y',Condiciones.shape[1],'columnas')

El DataFrame está compuesto por 483,952 filas y 1 columnas


In [ ]:
# 3. Eliminamos los duplicados
Condiciones = Condiciones.drop_duplicates()

In [ ]:
# 4. Eliminamos los valores vacíos
Condiciones = Condiciones.dropna()

In [ ]:
# 5. Reseteamos el ínidce y confimamos  de la Estructura del Dataset
Condiciones = Condiciones.reset_index(drop=True)
print('El DataFrame está compuesto por {:,}'.format(Condiciones.shape[0]),'filas y',Condiciones.shape[1],'columnas')

El DataFrame está compuesto por 8 filas y 1 columnas


In [ ]:
#6. Creamos codigo de condición
Condiciones['COD_CONDICION']=Condiciones.index+1

In [ ]:
print(Condiciones.head(8))

              CONDICIONES  COD_CONDICION
0                    SECA              1
1         MATERIAL SUELTO              2
2                  HUMEDA              3
3  ALCANTARILLA DESTAPADA              4
4                    LODO              5
5                  ACEITE              6
6                    OTRA              7
7       MATERIAL ORGANICO              8


In [ ]:
Condiciones["COD_CONDICION"] = pd.to_numeric(Condiciones["COD_CONDICION"] , downcast="integer", errors="coerce")
Condiciones["COD_CONDICION"] = Condiciones["COD_CONDICION"].astype(int)

In [ ]:
print('Las variables almacenan la información en el siguiente formato:', Condiciones.dtypes)

Las variables almacenan la información en el siguiente formato: CONDICIONES      object
COD_CONDICION     int64
dtype: object


### 3.10 Creación tabla Iluminacion A

In [ ]:
# 1. Crear la tabla de Iluminacion A con los campos necesarios
Iluminacion_A = pd.read_csv('/content/Via.csv')
Iluminacion_A = Iluminacion_A[['ILUMINACION_A']]

<ipython-input-105-2f01866532e0>:2: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  Iluminacion_A = pd.read_csv('/content/Via.csv')


In [ ]:
# 2. Revisión de la Estructura del Dataset
print('El DataFrame está compuesto por {:,}'.format(Iluminacion_A.shape[0]),'filas y',Iluminacion_A.shape[1],'columnas')

El DataFrame está compuesto por 483,952 filas y 1 columnas


In [ ]:
# 3. Eliminamos los duplicados
Iluminacion_A = Iluminacion_A.drop_duplicates()

In [ ]:
# 4. Eliminamos los valores vacíos
Iluminacion_A = Iluminacion_A.dropna()

In [ ]:
# 5. Reseteamos el ínidce y confimamos  de la Estructura del Dataset
Iluminacion_A = Iluminacion_A.reset_index(drop=True)
print('El DataFrame está compuesto por {:,}'.format(Iluminacion_A.shape[0]),'filas y',Iluminacion_A.shape[1],'columnas')

El DataFrame está compuesto por 2 filas y 1 columnas


In [ ]:
#6. Creamos codigo de iluminación
Iluminacion_A['COD_ILUMINACION_A']=Iluminacion_A.index+1

In [ ]:
print(Iluminacion_A.head())

  ILUMINACION_A  COD_ILUMINACION_A
0           SIN                  1
1           CON                  2


In [ ]:
Iluminacion_A["COD_ILUMINACION_A"] = pd.to_numeric(Iluminacion_A["COD_ILUMINACION_A"] , downcast="integer", errors="coerce")
Iluminacion_A["COD_ILUMINACION_A"] = Iluminacion_A["COD_ILUMINACION_A"].astype(int)

In [ ]:
print('Las variables almacenan la información en el siguiente formato:', Iluminacion_A.dtypes)

Las variables almacenan la información en el siguiente formato: ILUMINACION_A        object
COD_ILUMINACION_A     int64
dtype: object


### 3.11 Creación tabla Agente Transito

In [ ]:
# 1. Crear la tabla de Agente Transito con los campos necesarios
Agente_Transito = pd.read_csv('/content/Via.csv')
Agente_Transito = Agente_Transito[['AGENTE_TRANSITO']]

<ipython-input-114-e2ac2f5f9b89>:2: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  Agente_Transito = pd.read_csv('/content/Via.csv')


In [ ]:
# 2. Revisión de la Estructura del Dataset
print('El DataFrame está compuesto por {:,}'.format(Agente_Transito.shape[0]),'filas y',Agente_Transito.shape[1],'columnas')

El DataFrame está compuesto por 483,952 filas y 1 columnas


In [ ]:
# 3. Eliminamos los duplicados
Agente_Transito = Agente_Transito.drop_duplicates()

In [ ]:
# 4. Eliminamos los valores vacíos
Agente_Transito = Agente_Transito.dropna()

In [ ]:
# 5. Reseteamos el ínidce y confimamos  de la Estructura del Dataset
Agente_Transito = Agente_Transito.reset_index(drop=True)
print('El DataFrame está compuesto por {:,}'.format(Agente_Transito.shape[0]),'filas y',Agente_Transito.shape[1],'columnas')

El DataFrame está compuesto por 2 filas y 1 columnas


In [ ]:
#6. Creamos codigo de agente
Agente_Transito['COD_AGENTE_TRANSITO']=Agente_Transito.index+1

In [ ]:
print(Agente_Transito.head())

  AGENTE_TRANSITO  COD_AGENTE_TRANSITO
0              NO                    1
1              SI                    2


In [ ]:
Agente_Transito["COD_AGENTE_TRANSITO"] = pd.to_numeric(Agente_Transito["COD_AGENTE_TRANSITO"] , downcast="integer", errors="coerce")
Agente_Transito["COD_AGENTE_TRANSITO"] = Agente_Transito["COD_AGENTE_TRANSITO"].astype(int)

In [ ]:
print('Las variables almacenan la información en el siguiente formato:', Agente_Transito.dtypes)

Las variables almacenan la información en el siguiente formato: AGENTE_TRANSITO        object
COD_AGENTE_TRANSITO     int64
dtype: object


### 3.12 Creación tabla Semaforo

In [ ]:
# 1. Crear la tabla de Semaforo con los campos necesarios
Semaforo = pd.read_csv('/content/Via.csv')
Semaforo = Semaforo[['SEMAFORO']]

<ipython-input-123-b2b72246977a>:2: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  Semaforo = pd.read_csv('/content/Via.csv')


In [ ]:
# 2. Revisión de la Estructura del Dataset
print('El DataFrame está compuesto por {:,}'.format(Semaforo.shape[0]),'filas y',Semaforo.shape[1],'columnas')

El DataFrame está compuesto por 483,952 filas y 1 columnas


In [ ]:
# 3. Eliminamos los duplicados
Semaforo = Semaforo.drop_duplicates()

In [ ]:
# 4. Eliminamos los valores vacíos
Semaforo = Semaforo.dropna()

In [ ]:
# 5. Reseteamos el ínidce y confimamos  de la Estructura del Dataset
Semaforo = Semaforo.reset_index(drop=True)
print('El DataFrame está compuesto por {:,}'.format(Semaforo.shape[0]),'filas y',Semaforo.shape[1],'columnas')

El DataFrame está compuesto por 6 filas y 1 columnas


In [ ]:
#6. Creamos codigo de semaforo
Semaforo['COD_SEMAFORO']=Semaforo.index+1

In [ ]:
print(Semaforo.head(6))

       SEMAFORO  COD_SEMAFORO
0      OPERANDO             1
1        OCULTO             2
2  INTERMITENTE             3
3     CON DAÑOS             4
4       APAGADO             5
5       NINGUNO             6


In [ ]:
Semaforo["COD_SEMAFORO"] = pd.to_numeric(Semaforo["COD_SEMAFORO"] , downcast="integer", errors="coerce")
Semaforo["COD_SEMAFORO"] = Semaforo["COD_SEMAFORO"].astype(int)

In [ ]:
print('Las variables almacenan la información en el siguiente formato:', Semaforo.dtypes)

Las variables almacenan la información en el siguiente formato: SEMAFORO        object
COD_SEMAFORO     int64
dtype: object


### 3.13 Creación tabla Tiempo

In [ ]:
# 1. Crear la tabla de tiempo con los campos necesarios
Tiempo = pd.read_csv('/content/Accidente.csv')
Tiempo = Tiempo[['FECHA_OCURRENCIA_ACC']]

In [ ]:
# 2. Revisión de la Estructura del Dataset
print('El DataFrame está compuesto por {:,}'.format(Tiempo.shape[0]),'filas y',Tiempo.shape[1],'columnas')
print('Con información distribuida en las siguientes variables:', Tiempo.columns)
print('Las variables almacenan la información en el siguiente formato:', Tiempo.dtypes)

El DataFrame está compuesto por 472,767 filas y 1 columnas
Con información distribuida en las siguientes variables: Index(['FECHA_OCURRENCIA_ACC'], dtype='object')
Las variables almacenan la información en el siguiente formato: FECHA_OCURRENCIA_ACC    object
dtype: object


In [ ]:
# 3. Eliminamos los duplicados
Tiempo = Tiempo.drop_duplicates()

In [ ]:
# 4. Eliminamos los valores vacíos
Tiempo = Tiempo.dropna()

In [ ]:
# 5. Reseteamos el ínidce y confimamos  de la Estructura del Dataset
Tiempo = Tiempo.reset_index(drop=True)
print('El DataFrame está compuesto por {:,}'.format(Tiempo.shape[0]),'filas y',Tiempo.shape[1],'columnas')

El DataFrame está compuesto por 6,756 filas y 1 columnas


In [ ]:
# 6. Tranformamos la columna Fecha a formato fecha
Tiempo.loc[:, 'FECHA_OCURRENCIA_ACC'] = pd.to_datetime(Tiempo['FECHA_OCURRENCIA_ACC']).dt.strftime('%d/%m/%Y')
Tiempo["FECHA_OCURRENCIA_ACC"]=pd.to_datetime(Tiempo["FECHA_OCURRENCIA_ACC"], format="%d/%m/%Y")

In [ ]:
# 7. Creamos las columnas de año, nro_mes, txt_mes, nro_dia, txt_dia, trimestre para poblarlas a partir de la fecha
Tiempo['ANO_OCURRENCIA'] = ""
Tiempo['NRO_MES_OCURRENCIA'] = ""
Tiempo['TXT_MES_OCURRENCIA'] = ""
Tiempo['NRO_DIA_OCURRENCIA'] = ""
Tiempo['TXT_DIA_OCURRENCIA'] = ""
Tiempo['TRIMESTRE_OCURRENCIA'] = ""

In [ ]:
# 8. Poblamos las columnas creadas con la información de la columna de fecha de ocurrencia
for i in range(len(Tiempo['FECHA_OCURRENCIA_ACC'])):
  Tiempo.loc[i,"ANO_OCURRENCIA"]=(Tiempo.loc[i, "FECHA_OCURRENCIA_ACC"]).year

for i in range(len(Tiempo['FECHA_OCURRENCIA_ACC'])):
  Tiempo.loc[i,"NRO_MES_OCURRENCIA"]=(Tiempo.loc[i, "FECHA_OCURRENCIA_ACC"]).month

for i in range(len(Tiempo['FECHA_OCURRENCIA_ACC'])):
  Tiempo.loc[i,"TXT_MES_OCURRENCIA"]=(Tiempo.loc[i, "FECHA_OCURRENCIA_ACC"]).month_name()

for i in range(len(Tiempo['FECHA_OCURRENCIA_ACC'])):
  Tiempo.loc[i,"NRO_DIA_OCURRENCIA"]=(Tiempo.loc[i, "FECHA_OCURRENCIA_ACC"]).day

for i in range(len(Tiempo['FECHA_OCURRENCIA_ACC'])):
  Tiempo.loc[i,"TXT_DIA_OCURRENCIA"]=(Tiempo.loc[i, "FECHA_OCURRENCIA_ACC"]).day_name()

for i in range(len(Tiempo['FECHA_OCURRENCIA_ACC'])):
  Tiempo.loc[i,"TRIMESTRE_OCURRENCIA"]=(Tiempo.loc[i, "FECHA_OCURRENCIA_ACC"]).quarter

In [ ]:
#7. Se almacena solo la fecha de ocurrencia
Tiempo['FECHA_OCURRENCIA_ACC'] = Tiempo['FECHA_OCURRENCIA_ACC'].dt.date

In [ ]:
print(Tiempo.head())

  FECHA_OCURRENCIA_ACC ANO_OCURRENCIA NRO_MES_OCURRENCIA TXT_MES_OCURRENCIA  \
0           2017-06-12           2017                  6               June   
1           2022-10-29           2022                 10            October   
2           2012-12-15           2012                 12           December   
3           2022-11-01           2022                 11           November   
4           2014-10-26           2014                 10            October   

  NRO_DIA_OCURRENCIA TXT_DIA_OCURRENCIA TRIMESTRE_OCURRENCIA  
0                 12             Monday                    2  
1                 29           Saturday                    4  
2                 15           Saturday                    4  
3                  1            Tuesday                    4  
4                 26             Sunday                    4  


In [ ]:
# 9. Asegurar tipo de datos
Tiempo["ANO_OCURRENCIA"] = pd.to_numeric(Tiempo["ANO_OCURRENCIA"], downcast="integer", errors="coerce")
Tiempo["NRO_MES_OCURRENCIA"] = pd.to_numeric(Tiempo["NRO_MES_OCURRENCIA"], downcast="integer", errors="coerce")
Tiempo["NRO_DIA_OCURRENCIA"] = pd.to_numeric(Tiempo["NRO_DIA_OCURRENCIA"], downcast="integer", errors="coerce")
Tiempo["TRIMESTRE_OCURRENCIA"] = pd.to_numeric(Tiempo["TRIMESTRE_OCURRENCIA"], downcast="integer", errors="coerce")

In [ ]:
# 10. Contabilizamos valores nulos, e identificamos participación en el total de registros por columnas
resumen_NaN_Tiempo = pd.DataFrame(Tiempo.isnull().sum())
resumen_NaN_Tiempo.columns=['Nulos']
resumen_NaN_Tiempo['Registros']=Tiempo.shape[0]
resumen_NaN_Tiempo['% Nulos']=resumen_NaN_Tiempo.Nulos/resumen_NaN_Tiempo.Registros
resumen_NaN_Tiempo['% Nulos']=resumen_NaN_Tiempo['% Nulos'].apply("{:.2%}".format)
resumen_NaN_Tiempo

,Nulos,Registros,% Nulos
FECHA_OCURRENCIA_ACC,0,6756,0.00%
ANO_OCURRENCIA,0,6756,0.00%
NRO_MES_OCURRENCIA,0,6756,0.00%
TXT_MES_OCURRENCIA,0,6756,0.00%
NRO_DIA_OCURRENCIA,0,6756,0.00%
TXT_DIA_OCURRENCIA,0,6756,0.00%
TRIMESTRE_OCURRENCIA,0,6756,0.00%


In [ ]:
print('Las variables almacenan la información en el siguiente formato:', Tiempo.dtypes)

Las variables almacenan la información en el siguiente formato: FECHA_OCURRENCIA_ACC    object
ANO_OCURRENCIA           int16
NRO_MES_OCURRENCIA        int8
TXT_MES_OCURRENCIA      object
NRO_DIA_OCURRENCIA        int8
TXT_DIA_OCURRENCIA      object
TRIMESTRE_OCURRENCIA      int8
dtype: object


### 3.14 Creación tabla Horario

In [ ]:
# 1. Crear la tabla de horario con los campos necesarios
Horario = pd.read_csv('/content/Accidente.csv')
Horario = Horario[['HORA_OCURRENCIA_ACC']]

In [ ]:
# 2. Revisión de la Estructura del Dataset
print('El DataFrame está compuesto por {:,}'.format(Horario.shape[0]),'filas y',Horario.shape[1],'columnas')
print('Con información distribuida en las siguientes variables:', Horario.columns)
print('Las variables almacenan la información en el siguiente formato:', Horario.dtypes)

El DataFrame está compuesto por 472,767 filas y 1 columnas
Con información distribuida en las siguientes variables: Index(['HORA_OCURRENCIA_ACC'], dtype='object')
Las variables almacenan la información en el siguiente formato: HORA_OCURRENCIA_ACC    object
dtype: object


In [ ]:
# 3. Eliminamos los duplicados
Horario = Horario.drop_duplicates()

In [ ]:
# 4. Eliminamos los valores vacíos
Horario = Horario.dropna()

In [ ]:
# 5. Reseteamos el ínidce y confimamos  de la Estructura del Dataset
Horario = Horario.reset_index(drop=True)
print('El DataFrame está compuesto por {:,}'.format(Horario.shape[0]),'filas y',Horario.shape[1],'columnas')

El DataFrame está compuesto por 1,440 filas y 1 columnas


In [ ]:
# 6. Creamos las columna de jornada
Horario['HORA_OCURRENCIA'] = ""
Horario['JORNADA_OCURRENCIA'] = ""

In [ ]:
# 7. Poblamos las columnas creadas con la información de la columna de fecha de ocurrencia
Horario['HORA_OCURRENCIA'] = Horario['HORA_OCURRENCIA_ACC'].str.split(':').str[0]

Madrugada = ["00", "01", "02", "03", "04", "05"]
Mañana = ["06", "07", "08", "09", "10", "11"]
Tarde = ["12", "13", "14", "15", "16", "17"]
Noche = ["18", "19", "20", "21", "22", "23"]

for i in range(len(Horario['HORA_OCURRENCIA_ACC'])):
  if Horario.loc[i,"HORA_OCURRENCIA"] in Madrugada:
    Horario.loc[i,"JORNADA_OCURRENCIA"]="Madrugada"
  elif Horario.loc[i,"HORA_OCURRENCIA"] in Mañana:
    Horario.loc[i,"JORNADA_OCURRENCIA"]="Mañana"
  elif Horario.loc[i,"HORA_OCURRENCIA"] in Tarde:
    Horario.loc[i,"JORNADA_OCURRENCIA"]="Tarde"
  elif Horario.loc[i,"HORA_OCURRENCIA"] in Noche:
    Horario.loc[i,"JORNADA_OCURRENCIA"]="Noche"


In [ ]:
# 8. Contabilizamos valores nulos, e identificamos participación en el total de registros por columnas
resumen_NaN_Horario = pd.DataFrame(Horario.isnull().sum())
resumen_NaN_Horario.columns=['Nulos']
resumen_NaN_Horario['Registros']=Horario.shape[0]
resumen_NaN_Horario['% Nulos']=resumen_NaN_Horario.Nulos/resumen_NaN_Horario.Registros
resumen_NaN_Horario['% Nulos']=resumen_NaN_Horario['% Nulos'].apply("{:.2%}".format)
resumen_NaN_Horario

,Nulos,Registros,% Nulos
HORA_OCURRENCIA_ACC,0,1440,0.00%
HORA_OCURRENCIA,0,1440,0.00%
JORNADA_OCURRENCIA,0,1440,0.00%


In [ ]:
print(Horario.head())

  HORA_OCURRENCIA_ACC HORA_OCURRENCIA JORNADA_OCURRENCIA
0            05:30:00              05          Madrugada
1            10:55:00              10             Mañana
2            20:30:00              20              Noche
3            14:58:00              14              Tarde
4            21:25:00              21              Noche


In [ ]:
print('Las variables almacenan la información en el siguiente formato:', Horario.dtypes)

Las variables almacenan la información en el siguiente formato: HORA_OCURRENCIA_ACC    object
HORA_OCURRENCIA        object
JORNADA_OCURRENCIA     object
dtype: object


### 3.15 Creación tabla Gravedad

In [ ]:
# 1. Crear la tabla de Gravedad con los campos necesarios
Gravedad = pd.read_csv('/content/Accidente.csv')
Gravedad = Gravedad[['GRAVEDAD']]

In [ ]:
# 2. Revisión de la Estructura del Dataset
print('El DataFrame está compuesto por {:,}'.format(Gravedad.shape[0]),'filas y',Gravedad.shape[1],'columnas')

El DataFrame está compuesto por 472,767 filas y 1 columnas


In [ ]:
# 3. Eliminamos los duplicados
Gravedad = Gravedad.drop_duplicates()

In [ ]:
# 4. Eliminamos los valores vacíos
Gravedad = Gravedad.dropna()

In [ ]:
# 5. Reseteamos el ínidce y confimamos  de la Estructura del Dataset
Gravedad = Gravedad.reset_index(drop=True)
print('El DataFrame está compuesto por {:,}'.format(Gravedad.shape[0]),'filas y',Gravedad.shape[1],'columnas')

El DataFrame está compuesto por 3 filas y 1 columnas


In [ ]:
#6. Creamos codigo de gravedad
Gravedad['COD_GRAVEDAD']=Gravedad.index+1

In [ ]:
print(Gravedad.head())

      GRAVEDAD  COD_GRAVEDAD
0   SOLO DANOS             1
1  CON HERIDOS             2
2  CON MUERTOS             3


In [ ]:
Gravedad["COD_GRAVEDAD"] = pd.to_numeric(Gravedad["COD_GRAVEDAD"] , downcast="float", errors="coerce")
Gravedad["COD_GRAVEDAD"] = Gravedad["COD_GRAVEDAD"].astype(float)

In [ ]:
print('Las variables almacenan la información en el siguiente formato:', Gravedad.dtypes)

Las variables almacenan la información en el siguiente formato: GRAVEDAD         object
COD_GRAVEDAD    float64
dtype: object


### 3.16 Creación tabla Clase_Accidente

In [ ]:
# 1. Crear la tabla de Clase_Accidente con los campos necesarios
Clase_Accidente = pd.read_csv('/content/Accidente.csv')
Clase_Accidente = Clase_Accidente[['CLASE_ACC']]

In [ ]:
# 2. Revisión de la Estructura del Dataset
print('El DataFrame está compuesto por {:,}'.format(Clase_Accidente.shape[0]),'filas y',Clase_Accidente.shape[1],'columnas')

El DataFrame está compuesto por 472,767 filas y 1 columnas


In [ ]:
# 3. Eliminamos los duplicados
Clase_Accidente = Clase_Accidente.drop_duplicates()

In [ ]:
# 4. Eliminamos los valores vacíos
Clase_Accidente = Clase_Accidente.dropna()

In [ ]:
# 4. Reseteamos el ínidce y confimamos  de la Estructura del Dataset
Clase_Accidente = Clase_Accidente.reset_index(drop=True)
print('El DataFrame está compuesto por {:,}'.format(Clase_Accidente.shape[0]),'filas y',Clase_Accidente.shape[1],'columnas')

El DataFrame está compuesto por 7 filas y 1 columnas


In [ ]:
#6. Creamos codigo de Clase_Accidente
Clase_Accidente['COD_CLASE_ACC']=Clase_Accidente.index+1

In [ ]:
print(Clase_Accidente.head(7))

           CLASE_ACC  COD_CLASE_ACC
0             CHOQUE              1
1          ATROPELLO              2
2         AUTOLESION              3
3        VOLCAMIENTO              4
4               OTRO              5
5  CAIDA DE OCUPANTE              6
6           INCENDIO              7


In [ ]:
Clase_Accidente["COD_CLASE_ACC"] = pd.to_numeric(Clase_Accidente["COD_CLASE_ACC"] , downcast="float", errors="coerce")
Clase_Accidente["COD_CLASE_ACC"] = Clase_Accidente["COD_CLASE_ACC"].astype(float)

In [ ]:
print('Las variables almacenan la información en el siguiente formato:', Clase_Accidente.dtypes)

Las variables almacenan la información en el siguiente formato: CLASE_ACC         object
COD_CLASE_ACC    float64
dtype: object


### 3.17 Creación tabla Localidad

In [ ]:
# 1. Crear la tabla de Localidad con los campos necesarios
Localidad = pd.read_csv('/content/Accidente.csv')
Localidad = Localidad[['LOCALIDAD']]

In [ ]:
# 2. Revisión de la Estructura del Dataset
print('El DataFrame está compuesto por {:,}'.format(Localidad.shape[0]),'filas y',Localidad.shape[1],'columnas')

El DataFrame está compuesto por 472,767 filas y 1 columnas


In [ ]:
# 3. Eliminamos los duplicados
Localidad = Localidad.drop_duplicates()

In [ ]:
# 4. Eliminamos los valores vacíos
Localidad = Localidad.dropna()

In [ ]:
# 4. Reseteamos el ínidce y confimamos  de la Estructura del Dataset
Localidad = Localidad.reset_index(drop=True)
print('El DataFrame está compuesto por {:,}'.format(Localidad.shape[0]),'filas y',Localidad.shape[1],'columnas')

El DataFrame está compuesto por 20 filas y 1 columnas


In [ ]:
#6. Creamos codigo de Localidad
Localidad['COD_LOCALIDAD']=Localidad.index+1

In [ ]:
print(Localidad.head(20))

             LOCALIDAD  COD_LOCALIDAD
0             ENGATIVA              1
1            CHAPINERO              2
2                 SUBA              3
3             FONTIBON              4
4       CIUDAD BOLIVAR              5
5         LOS MARTIRES              6
6             SANTA FE              7
7                 BOSA              8
8              USAQUEN              9
9              KENNEDY             10
10  RAFAEL URIBE URIBE             11
11         TEUSAQUILLO             12
12       SAN CRISTOBAL             13
13       PUENTE ARANDA             14
14          TUNJUELITO             15
15      ANTONIO NARINO             16
16                USME             17
17      BARRIOS UNIDOS             18
18          CANDELARIA             19
19             SUMAPAZ             20


In [ ]:
Localidad["COD_LOCALIDAD"] = pd.to_numeric(Localidad["COD_LOCALIDAD"] , downcast="float", errors="coerce")
Localidad["COD_LOCALIDAD"] = Localidad["COD_LOCALIDAD"].astype(float)

In [ ]:
print('Las variables almacenan la información en el siguiente formato:', Localidad.dtypes)

Las variables almacenan la información en el siguiente formato: LOCALIDAD         object
COD_LOCALIDAD    float64
dtype: object


### 3.18 Creación tabla Descripcion_Causa

In [ ]:
# 1. Crear la tabla de Describe_Causa con los campos necesarios
Describe_Causa = pd.read_csv('/content/Causa.csv')
Describe_Causa = Describe_Causa[['CODIGO_CAUSA', 'NOMBRE']]

<ipython-input-182-40184f6d4205>:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  Describe_Causa = pd.read_csv('/content/Causa.csv')


In [ ]:
# 2. Revisión de la Estructura del Dataset
print('El DataFrame está compuesto por {:,}'.format(Describe_Causa.shape[0]),'filas y',Describe_Causa.shape[1],'columnas')

El DataFrame está compuesto por 682,392 filas y 2 columnas


In [ ]:
# 3. Obligamos a que el código causa sea interpretado como número
Describe_Causa["CODIGO_CAUSA"] = pd.to_numeric(Describe_Causa["CODIGO_CAUSA"] , downcast="float", errors="coerce")
Describe_Causa["CODIGO_CAUSA"] = Describe_Causa["CODIGO_CAUSA"].astype(float)

In [ ]:
# 4. Eliminamos los duplicados
Describe_Causa = Describe_Causa.drop_duplicates()

In [ ]:
# 5. Eliminamos los valores vacíos
Describe_Causa = Describe_Causa.dropna()

In [ ]:
# 6. Reseteamos el ínidce y confirmamos la Estructura del Dataset
Describe_Causa = Describe_Causa.reset_index(drop=True)
print('El DataFrame está compuesto por {:,}'.format(Describe_Causa.shape[0]),'filas y',Describe_Causa.shape[1],'columnas')

El DataFrame está compuesto por 110 filas y 2 columnas


In [ ]:
print(Describe_Causa.head(6))

   CODIGO_CAUSA                                        NOMBRE
0         157.0                                          OTRA
1         121.0            NO MANTENER DISTANCIA DE SEGURIDAD
2         143.0  PONER EN MARCHA UN VEHICULO SIN PRECAUCIONES
3         139.0          NO UTILIZAR - IMPERICIA EN EL MANEJO
4         103.0                            ADELANTAR CERRANDO
5         104.0                      ADELANTAR INVADIENDO VIA


In [ ]:
print('Las variables almacenan la información en el siguiente formato:', Describe_Causa.dtypes)

Las variables almacenan la información en el siguiente formato: CODIGO_CAUSA    float64
NOMBRE           object
dtype: object


### 3.19 Creación tabla Tipo_Causa

In [ ]:
# 1. Crear la tabla de Tipo_Causa con los campos necesarios
Tipo_Causa = pd.read_csv('/content/Causa.csv')
Tipo_Causa = Tipo_Causa[['TIPO', 'TIPO_CAUSA']]

<ipython-input-190-07d70b2b9ea2>:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  Tipo_Causa = pd.read_csv('/content/Causa.csv')


In [ ]:
# 2. Revisión de la Estructura del Dataset
print('El DataFrame está compuesto por {:,}'.format(Tipo_Causa.shape[0]),'filas y',Tipo_Causa.shape[1],'columnas')

El DataFrame está compuesto por 682,392 filas y 2 columnas


In [ ]:
# 3. Eliminamos los duplicados
Tipo_Causa = Tipo_Causa.drop_duplicates()

In [ ]:
# 4. Eliminamos los valores vacíos
Tipo_Causa = Tipo_Causa.dropna()

In [ ]:
# 5. Reseteamos el ínidce y confirmamos la Estructura del Dataset
Tipo_Causa = Tipo_Causa.reset_index(drop=True)
print('El DataFrame está compuesto por {:,}'.format(Tipo_Causa.shape[0]),'filas y',Tipo_Causa.shape[1],'columnas')

El DataFrame está compuesto por 6 filas y 2 columnas


In [ ]:
print(Tipo_Causa.head(6))

  TIPO TIPO_CAUSA
0   CG  CONDUCTOR
1   VI        VIA
2   CE  CONDUCTOR
3   VH   VEHICULO
4   PE     PEATON
5   PA   PASAJERO


In [ ]:
print('Las variables almacenan la información en el siguiente formato:', Tipo_Causa.dtypes)

Las variables almacenan la información en el siguiente formato: TIPO          object
TIPO_CAUSA    object
dtype: object


### 3.20 Creación tabla Clase_Vehiculo

In [ ]:
# 1. Crear la tabla de Tipo_Causa con los campos necesarios
Clase_Vh = pd.read_csv('/content/Vehiculo.csv')
Clase_Vh = Clase_Vh[['CLASE']]

<ipython-input-197-6579a86053bf>:2: DtypeWarning: Columns (1,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  Clase_Vh = pd.read_csv('/content/Vehiculo.csv')


In [ ]:
# 2. Revisión de la Estructura del Dataset
print('El DataFrame está compuesto por {:,}'.format(Clase_Vh.shape[0]),'filas y',Clase_Vh.shape[1],'columnas')

El DataFrame está compuesto por 1,441,024 filas y 1 columnas


In [ ]:
# 3. Eliminamos los duplicados
Clase_Vh = Clase_Vh.drop_duplicates()

In [ ]:
# 4. Eliminamos los valores vacíos
Clase_Vh = Clase_Vh.dropna()

In [ ]:
# 5. Reseteamos el ínidce y confirmamos la Estructura del Dataset
Clase_Vh = Clase_Vh.reset_index(drop=True)
print('El DataFrame está compuesto por {:,}'.format(Clase_Vh.shape[0]),'filas y',Clase_Vh.shape[1],'columnas')

El DataFrame está compuesto por 27 filas y 1 columnas


In [ ]:
# 6. Creamos codigo de Clase
Clase_Vh['COD_CLASE']=Clase_Vh.index+1

In [ ]:
# 7. Corregir el registro en el que dice AMBULACIA por AMBULANCIA
if 'AMBULACIA' in Clase_Vh['CLASE'].values:
  Clase_Vh['CLASE'] = Clase_Vh['CLASE'].replace('AMBULACIA', 'AMBULANCIA')

In [ ]:
print(Clase_Vh.head(27))

              CLASE  COD_CLASE
0       MOTOCICLETA          1
1         AUTOMOVIL          2
2               BUS          3
3      TRACTOCAMION          4
4    CAMION, FURGON          5
5         CAMIONETA          6
6          MICROBUS          7
7           CAMPERO          8
8            BUSETA          9
9          VOLQUETA         10
10        MOTOCARRO         11
11        BICICLETA         12
12  NO IDENTIFICADO         13
13  TRACCION ANIMAL         14
14             OTRO         15
15        MOTOCICLO         16
16    M. INDUSTRIAL         17
17      M. AGRICOLA         18
18   BUS ARTICULADO         19
19  BUS ALIMENTADOR         20
20            METRO         21
21       AMBULANCIA         22
22             TREN         23
23         BICITAXI         24
24       CUATRIMOTO         25
25         REMOLQUE         26
26    SEMI-REMOLQUE         27


In [ ]:
Clase_Vh["COD_CLASE"] = pd.to_numeric(Clase_Vh["COD_CLASE"] , downcast="float", errors="coerce")
Clase_Vh["COD_CLASE"] = Clase_Vh["COD_CLASE"].astype(float)

In [ ]:
print('Las variables almacenan la información en el siguiente formato:', Clase_Vh.dtypes)

Las variables almacenan la información en el siguiente formato: CLASE         object
COD_CLASE    float64
dtype: object


### 3.21 Creación tabla Servicio

In [ ]:
# 1. Crear la tabla de Servicio con los campos necesarios
Servicio = pd.read_csv('/content/Vehiculo.csv')
Servicio = Servicio[['SERVICIO']]

<ipython-input-207-33956d71d8ba>:2: DtypeWarning: Columns (1,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  Servicio = pd.read_csv('/content/Vehiculo.csv')


In [ ]:
# 2. Revisión de la Estructura del Dataset
print('El DataFrame está compuesto por {:,}'.format(Servicio.shape[0]),'filas y',Servicio.shape[1],'columnas')

El DataFrame está compuesto por 1,441,024 filas y 1 columnas


In [ ]:
# 3. Eliminamos los duplicados
Servicio = Servicio.drop_duplicates()

In [ ]:
# 4. Eliminamos los valores vacíos
Servicio = Servicio.dropna()

In [ ]:
# 5. Reseteamos el ínidce y confirmamos la Estructura del Dataset
Servicio = Servicio.reset_index(drop=True)
print('El DataFrame está compuesto por {:,}'.format(Servicio.shape[0]),'filas y',Servicio.shape[1],'columnas')

El DataFrame está compuesto por 5 filas y 1 columnas


In [ ]:
# 6. Creamos codigo de servicio
Servicio['COD_SERVICIO']=Servicio.index+1

In [ ]:
# 7. Eliminar los espacios en balnco al inicio y al final
Servicio['SERVICIO'] = Servicio['SERVICIO'].str.strip()

In [ ]:
print(Servicio.head(5))

          SERVICIO  COD_SERVICIO
0       PARTICULAR             1
1          PUBLICO             2
2          OFICIAL             3
3      DIPLOMATICO             4
4  SIN INFORMACION             5


In [ ]:
Servicio["COD_SERVICIO"] = pd.to_numeric(Servicio["COD_SERVICIO"] , downcast="float", errors="coerce")
Servicio["COD_SERVICIO"] = Servicio["COD_SERVICIO"].astype(float)

In [ ]:
print('Las variables almacenan la información en el siguiente formato:', Servicio.dtypes)

Las variables almacenan la información en el siguiente formato: SERVICIO         object
COD_SERVICIO    float64
dtype: object


### 3.22 Creación tabla Modalidad

In [ ]:
# 1. Crear la tabla de Modalidad con los campos necesarios
Modalidad = pd.read_csv('/content/Vehiculo.csv')
Modalidad = Modalidad[['MODALIDAD']]

<ipython-input-217-1876360564d5>:2: DtypeWarning: Columns (1,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  Modalidad = pd.read_csv('/content/Vehiculo.csv')


In [ ]:
# 2. Revisión de la Estructura del Dataset
print('El DataFrame está compuesto por {:,}'.format(Modalidad.shape[0]),'filas y',Modalidad.shape[1],'columnas')

El DataFrame está compuesto por 1,441,024 filas y 1 columnas


In [ ]:
# 3. Eliminamos los duplicados
Modalidad = Modalidad.drop_duplicates()

In [ ]:
# 4. Eliminamos los valores vacíos
Modalidad = Modalidad.dropna()

In [ ]:
# 5. Reseteamos el ínidce y confirmamos la Estructura del Dataset
Modalidad = Modalidad.reset_index(drop=True)
print('El DataFrame está compuesto por {:,}'.format(Modalidad.shape[0]),'filas y',Modalidad.shape[1],'columnas')

El DataFrame está compuesto por 12 filas y 1 columnas


In [ ]:
# 6. Creamos codigo de Modalidad
Modalidad['COD_MODALIDAD']=Modalidad.index+1

In [ ]:
print(Modalidad.head(12))

                         MODALIDAD  COD_MODALIDAD
0               PASAJEROS - MASIVO              1
1      CARGA - MERCANCIA PELIGROSA              2
2                     CARGA-PESADA              3
3           PASAJEROS - INDIVIDUAL              4
4            PASAJEROS - COLECTIVO              5
5     PASAJEROS - ESPECIAL TURISMO              6
6   PASAJEROS - ESPECIAL OCASIONAL              7
7                            MIXTO              8
8     PASAJEROS - ESPECIAL ESCOLAR              9
9              CARGA - EXTRAPESADA             10
10  PASAJEROS - ESPECIAL ASALARIAD             11
11       CARGA - EXTRADIMENSIONADA             12


In [ ]:
Modalidad["COD_MODALIDAD"] = pd.to_numeric(Modalidad["COD_MODALIDAD"] , downcast="float", errors="coerce")
Modalidad["COD_MODALIDAD"] = Modalidad["COD_MODALIDAD"].astype(float)

In [ ]:
print('Las variables almacenan la información en el siguiente formato:', Modalidad.dtypes)

Las variables almacenan la información en el siguiente formato: MODALIDAD         object
COD_MODALIDAD    float64
dtype: object


### 3.23 Creación de tabla Accidente

In [ ]:
Accidente = pd.read_csv('/content/Accidente.csv')

In [ ]:
Accidente.head()

,X,Y,OBJECTID,FORMULARIO,CODIGO_ACCIDENTE,FECHA_OCURRENCIA_ACC,HORA_OCURRENCIA_ACC,ANO_OCURRENCIA_ACC,MES_OCURRENCIA_ACC,DIA_OCURRENCIA_ACC,DIRECCION,GRAVEDAD,CLASE_ACC,LOCALIDAD,MUNICIPIO,FECHA_HORA_ACC,LATITUD,LONGITUD,CIV,PK_CALZADA
0,-74.090924,4.693807,1,A000640275,4484660,2017/06/12 00:00:00+00,05:30:00,2017,JUNIO,LUNES,AV AVENIDA BOYACA-CL 79 02,SOLO DANOS,CHOQUE,ENGATIVA,BOGOTA DC,2017/06/12 05:30:00+00,4.693807,-74.090924,10006772.0,221236.0
1,-74.119697,4.717638,2,A001515603,10583520,2022/10/29 00:00:00+00,10:55:00,2022,OCTUBRE,SABADO,KR 111 A - CL 79 02,SOLO DANOS,CHOQUE,ENGATIVA,BOGOTA DC,2022/10/29 10:55:00+00,4.717638,-74.119697,10000591.0,NaN
2,-74.040677,4.669288,3,A1241594,449558,2012/12/15 00:00:00+00,20:30:00,2012,DICIEMBRE,SABADO,KR 4-CL 91 02,SOLO DANOS,CHOQUE,CHAPINERO,BOGOTA DC,2012/12/15 20:30:00+00,4.669288,-74.040677,2002328.0,35683.0
3,-74.066101,4.737015,4,A001516620,10583513,2022/11/01 00:00:00+00,14:58:00,2022,NOVIEMBRE,MARTES,KR 72 - CL 148 02,CON HERIDOS,CHOQUE,SUBA,BOGOTA DC,2022/11/01 14:58:00+00,4.737015,-74.066101,11005846.0,NaN
4,-74.087407,4.679568,5,A000035070,513150,2014/10/26 00:00:00+00,21:25:00,2014,OCTUBRE,DOMINGO,AC 72-KR 68F 02,CON HERIDOS,ATROPELLO,ENGATIVA,BOGOTA DC,2014/10/26 21:25:00+00,4.679568,-74.087407,10008600.0,43287.0


In [ ]:
# 1. Revisión de la Estructura del Dataset
print('El DataFrame está compuesto por {:,}'.format(Accidente.shape[0]),'filas y',Accidente.shape[1],'columnas')
print('Con información distribuida en las siguientes variables:', Accidente.columns)
print('Las variables almacenan la información en el siguiente formato:', Accidente.dtypes)

El DataFrame está compuesto por 472,767 filas y 20 columnas
Con información distribuida en las siguientes variables: Index(['X', 'Y', 'OBJECTID', 'FORMULARIO', 'CODIGO_ACCIDENTE',
       'FECHA_OCURRENCIA_ACC', 'HORA_OCURRENCIA_ACC', 'ANO_OCURRENCIA_ACC',
       'MES_OCURRENCIA_ACC', 'DIA_OCURRENCIA_ACC', 'DIRECCION', 'GRAVEDAD',
       'CLASE_ACC', 'LOCALIDAD', 'MUNICIPIO', 'FECHA_HORA_ACC', 'LATITUD',
       'LONGITUD', 'CIV', 'PK_CALZADA'],
      dtype='object')
Las variables almacenan la información en el siguiente formato: X                       float64
Y                       float64
OBJECTID                  int64
FORMULARIO               object
CODIGO_ACCIDENTE          int64
FECHA_OCURRENCIA_ACC     object
HORA_OCURRENCIA_ACC      object
ANO_OCURRENCIA_ACC        int64
MES_OCURRENCIA_ACC       object
DIA_OCURRENCIA_ACC       object
DIRECCION                object
GRAVEDAD                 object
CLASE_ACC                object
LOCALIDAD                object
MUNICIPIO         

In [ ]:
# 2. Exploración de los Datos
description = Accidente.describe(include='all')
description

,X,Y,OBJECTID,FORMULARIO,CODIGO_ACCIDENTE,FECHA_OCURRENCIA_ACC,HORA_OCURRENCIA_ACC,ANO_OCURRENCIA_ACC,MES_OCURRENCIA_ACC,DIA_OCURRENCIA_ACC,DIRECCION,GRAVEDAD,CLASE_ACC,LOCALIDAD,MUNICIPIO,FECHA_HORA_ACC,LATITUD,LONGITUD,CIV,PK_CALZADA
count,472767.000000,472767.000000,472767.000000,472767,4.727670e+05,472767,472767,472767.000000,472767,472767,472767,472764,472627,472717,472767,472767,472767.000000,472767.000000,4.690330e+05,3.801120e+05
unique,NaN,NaN,NaN,472767,NaN,6756,1440,NaN,12,7,241171,3,7,20,1,382331,NaN,NaN,NaN,NaN
top,NaN,NaN,NaN,A000640275,NaN,2019/12/06 00:00:00+00,14:00:00,NaN,MARZO,VIERNES,AV AVENIDA BOYACA-CL 80 02,SOLO DANOS,CHOQUE,KENNEDY,BOGOTA DC,2010/09/17 18:30:00+00,NaN,NaN,NaN,NaN
freq,NaN,NaN,NaN,1,NaN,152,6280,NaN,41606,76051,416,295613,394196,54790,472767,8,NaN,NaN,NaN,NaN
mean,-74.101774,4.650681,236384.000000,NaN,4.444574e+06,NaN,NaN,2014.815082,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.650681,-74.101774,1.341007e+07,7.252240e+06
std,0.039927,0.056136,136476.221694,NaN,4.387758e+06,NaN,NaN,4.715196,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.056136,0.039927,1.319298e+07,1.820267e+07
min,-74.228300,4.085800,1.000000,NaN,1.040000e+02,NaN,NaN,2002.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.085800,-74.228300,0.000000e+00,0.000000e+00
25%,-74.131000,4.610782,118192.500000,NaN,3.930305e+05,NaN,NaN,2011.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.610782,-74.131000,7.001196e+06,4.213600e+04
50%,-74.100162,4.647700,236384.000000,NaN,4.417165e+06,NaN,NaN,2015.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.647700,-74.100162,1.000749e+07,1.755900e+05
75%,-74.070656,4.690738,354575.500000,NaN,1.047882e+07,NaN,NaN,2019.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.690738,-74.070656,1.400180e+07,2.331255e+05


In [ ]:
# 3. Contabilizamos valores nulos, e identificamos participación en el total de registros por columnas
resumen_NaN_Accidente = pd.DataFrame(Accidente.isnull().sum())
resumen_NaN_Accidente.columns=['Nulos']
resumen_NaN_Accidente['Registros']=Accidente.shape[0]
resumen_NaN_Accidente['% Nulos']=resumen_NaN_Accidente.Nulos/resumen_NaN_Accidente.Registros
resumen_NaN_Accidente['% Nulos']=resumen_NaN_Accidente['% Nulos'].apply("{:.2%}".format)
resumen_NaN_Accidente

,Nulos,Registros,% Nulos
X,0,472767,0.00%
Y,0,472767,0.00%
OBJECTID,0,472767,0.00%
FORMULARIO,0,472767,0.00%
CODIGO_ACCIDENTE,0,472767,0.00%
FECHA_OCURRENCIA_ACC,0,472767,0.00%
HORA_OCURRENCIA_ACC,0,472767,0.00%
ANO_OCURRENCIA_ACC,0,472767,0.00%
MES_OCURRENCIA_ACC,0,472767,0.00%
DIA_OCURRENCIA_ACC,0,472767,0.00%


In [ ]:
# 4. Cambio formato de campo FECHA_OCURRENCIA_ACC
Accidente.loc[:, 'FECHA_OCURRENCIA_ACC'] = pd.to_datetime(Accidente['FECHA_OCURRENCIA_ACC']).dt.strftime('%d/%m/%Y')
Accidente["FECHA_OCURRENCIA_ACC"]=pd.to_datetime(Accidente["FECHA_OCURRENCIA_ACC"], format="%d/%m/%Y")


In [ ]:
# 5. Traemos los codigos de localidad, gravedad, clase_acc que corresponde al indice de su tabla
Accidente = pd.merge(Accidente,Gravedad, how="left", left_on="GRAVEDAD", right_on="GRAVEDAD")
Accidente = pd.merge(Accidente,Clase_Accidente, how="left", left_on="CLASE_ACC", right_on="CLASE_ACC")
Accidente = pd.merge(Accidente,Localidad, how="left", left_on="LOCALIDAD", right_on="LOCALIDAD")

In [ ]:
print(Accidente.head())

           X         Y  OBJECTID  FORMULARIO  CODIGO_ACCIDENTE  \
0 -74.090924  4.693807         1  A000640275           4484660   
1 -74.119697  4.717638         2  A001515603          10583520   
2 -74.040677  4.669288         3    A1241594            449558   
3 -74.066101  4.737015         4  A001516620          10583513   
4 -74.087407  4.679568         5  A000035070            513150   

  FECHA_OCURRENCIA_ACC HORA_OCURRENCIA_ACC  ANO_OCURRENCIA_ACC  \
0           2017-06-12            05:30:00                2017   
1           2022-10-29            10:55:00                2022   
2           2012-12-15            20:30:00                2012   
3           2022-11-01            14:58:00                2022   
4           2014-10-26            21:25:00                2014   

  MES_OCURRENCIA_ACC DIA_OCURRENCIA_ACC  ...  LOCALIDAD  MUNICIPIO  \
0              JUNIO              LUNES  ...   ENGATIVA  BOGOTA DC   
1            OCTUBRE             SABADO  ...   ENGATIVA  BOGOTA DC

In [ ]:
# 6. Crear un nuevo DataFrame con las columnas seleccionadas
selected_columns = ['CODIGO_ACCIDENTE', 'FECHA_OCURRENCIA_ACC', 'HORA_OCURRENCIA_ACC', 'DIRECCION', 'COD_GRAVEDAD', 'COD_CLASE_ACC', 'COD_LOCALIDAD', 'LATITUD', 'LONGITUD']

# Asignar solo las columnas seleccionadas al nuevo DataFrame
Accidente_df = Accidente[selected_columns]
Accidente_df = pd.DataFrame(Accidente_df)

In [ ]:
# 7. Identificar registros duplicados
duplicates = Accidente_df.duplicated()

# Contar la cantidad de registros duplicados
num_duplicates = sum(duplicates)

print("Número de registros duplicados:", num_duplicates)
print("Registros duplicados:")
print(Accidente_df[duplicates])


Número de registros duplicados: 0
Registros duplicados:
Empty DataFrame
Columns: [CODIGO_ACCIDENTE, FECHA_OCURRENCIA_ACC, HORA_OCURRENCIA_ACC, DIRECCION, COD_GRAVEDAD, COD_CLASE_ACC, COD_LOCALIDAD, LATITUD, LONGITUD]
Index: []


In [ ]:
# Almacenamos solo la fecha de ocurrencia
Accidente_df['FECHA_OCURRENCIA_ACC'] = Accidente_df['FECHA_OCURRENCIA_ACC'].dt.date

In [ ]:
print('Las variables almacenan la información en el siguiente formato:', Accidente_df.dtypes)

Las variables almacenan la información en el siguiente formato: CODIGO_ACCIDENTE          int64
FECHA_OCURRENCIA_ACC     object
HORA_OCURRENCIA_ACC      object
DIRECCION                object
COD_GRAVEDAD            float64
COD_CLASE_ACC           float64
COD_LOCALIDAD           float64
LATITUD                 float64
LONGITUD                float64
dtype: object


In [ ]:
# Visualizar el nuevo DataFrame con las columnas seleccionadas
print(Accidente_df.head())

   CODIGO_ACCIDENTE FECHA_OCURRENCIA_ACC HORA_OCURRENCIA_ACC  \
0           4484660           2017-06-12            05:30:00   
1          10583520           2022-10-29            10:55:00   
2            449558           2012-12-15            20:30:00   
3          10583513           2022-11-01            14:58:00   
4            513150           2014-10-26            21:25:00   

                    DIRECCION  COD_GRAVEDAD  COD_CLASE_ACC  COD_LOCALIDAD  \
0  AV AVENIDA BOYACA-CL 79 02           1.0            1.0            1.0   
1      KR 111 A  - CL 79   02           1.0            1.0            1.0   
2               KR 4-CL 91 02           1.0            1.0            2.0   
3        KR 72  - CL 148   02           2.0            1.0            3.0   
4             AC 72-KR 68F 02           2.0            2.0            1.0   

    LATITUD   LONGITUD  
0  4.693807 -74.090924  
1  4.717638 -74.119697  
2  4.669288 -74.040677  
3  4.737015 -74.066101  
4  4.679568 -74.087407  


### 3.24 Creación tabla Condición_Actor

In [ ]:
# 1. Crear la tabla de Condicion_Actor con los campos necesarios
Condicion_Actor = pd.read_csv('/content/ActorVial.csv')
Condicion_Actor = Condicion_Actor[['CONDICION']]

<ipython-input-239-5ca4f32fd44d>:2: DtypeWarning: Columns (1,8) have mixed types. Specify dtype option on import or set low_memory=False.
  Condicion_Actor = pd.read_csv('/content/ActorVial.csv')


In [ ]:
# 2. Revisión de la Estructura del Dataset
print('El DataFrame está compuesto por {:,}'.format(Condicion_Actor.shape[0]),'filas y',Condicion_Actor.shape[1],'columnas')

El DataFrame está compuesto por 1,676,598 filas y 1 columnas


In [ ]:
# 3. Eliminamos los duplicados
Condicion_Actor = Condicion_Actor.drop_duplicates()

In [ ]:
# 4. Eliminamos los valores vacíos
Condicion_Actor = Condicion_Actor.dropna()

In [ ]:
# 5. Reseteamos el ínidce y confirmamos la Estructura del Dataset
Condicion_Actor = Condicion_Actor.reset_index(drop=True)
print('El DataFrame está compuesto por {:,}'.format(Condicion_Actor.shape[0]),'filas y',Condicion_Actor.shape[1],'columnas')

El DataFrame está compuesto por 6 filas y 1 columnas


In [ ]:
#6. Creamos codigo de condicion_actor
Condicion_Actor['COD_CONDICION_ACTOR']=Condicion_Actor.index+1

In [ ]:
print(Condicion_Actor.head(6))

         CONDICION  COD_CONDICION_ACTOR
0        CONDUCTOR                    1
1     MOTOCICLISTA                    2
2         CICLISTA                    3
3         PASAJERO                    4
4           PEATON                    5
5  SIN INFORMACION                    6


In [ ]:
Condicion_Actor["COD_CONDICION_ACTOR"] = pd.to_numeric(Condicion_Actor["COD_CONDICION_ACTOR"] , downcast="float", errors="coerce")
Condicion_Actor["COD_CONDICION_ACTOR"] = Condicion_Actor["COD_CONDICION_ACTOR"].astype(float)

In [ ]:
print('Las variables almacenan la información en el siguiente formato:', Condicion_Actor.dtypes)

Las variables almacenan la información en el siguiente formato: CONDICION               object
COD_CONDICION_ACTOR    float64
dtype: object


### 3.25 Creación tabla Estado

In [ ]:
# 1. Crear la tabla de Estado con los campos necesarios
Estado_ActorVial = pd.read_csv('/content/ActorVial.csv')
Estado_ActorVial = Estado_ActorVial[['ESTADO']]

<ipython-input-248-863d8ab0a836>:2: DtypeWarning: Columns (1,8) have mixed types. Specify dtype option on import or set low_memory=False.
  Estado_ActorVial = pd.read_csv('/content/ActorVial.csv')


In [ ]:
# 2. Revisión de la Estructura del Dataset
print('El DataFrame está compuesto por {:,}'.format(Estado_ActorVial.shape[0]),'filas y',Estado_ActorVial.shape[1],'columnas')

El DataFrame está compuesto por 1,676,598 filas y 1 columnas


In [ ]:
# 3. Eliminamos los duplicados
Estado_ActorVial = Estado_ActorVial.drop_duplicates()

In [ ]:
# 4. Eliminamos los valores vacíos
Estado_ActorVial = Estado_ActorVial.dropna()

In [ ]:
# 5. Reseteamos el ínidce y confirmamos la Estructura del Dataset
Estado_ActorVial = Estado_ActorVial.reset_index(drop=True)
print('El DataFrame está compuesto por {:,}'.format(Estado_ActorVial.shape[0]),'filas y',Estado_ActorVial.shape[1],'columnas')

El DataFrame está compuesto por 3 filas y 1 columnas


In [ ]:
#6. Creamos codigo de geometrica A
Estado_ActorVial['COD_ESTADO']=Estado_ActorVial.index+1

In [ ]:
print(Estado_ActorVial.head())

   ESTADO  COD_ESTADO
0   ILESO           1
1  HERIDO           2
2  MUERTO           3


In [ ]:
Estado_ActorVial["COD_ESTADO"] = pd.to_numeric(Estado_ActorVial["COD_ESTADO"] , downcast="float", errors="coerce")
Estado_ActorVial["COD_ESTADO"] = Estado_ActorVial["COD_ESTADO"].astype(float)

In [ ]:
print('Las variables almacenan la información en el siguiente formato:', Estado_ActorVial.dtypes)

Las variables almacenan la información en el siguiente formato: ESTADO         object
COD_ESTADO    float64
dtype: object


### 3.26 Creación tabla Genero

In [ ]:
# 1. Crear la tabla de Genero con los campos necesarios
Genero = pd.read_csv('/content/ActorVial.csv')
Genero = Genero[['GENERO']]

<ipython-input-257-718f8c3b76f9>:2: DtypeWarning: Columns (1,8) have mixed types. Specify dtype option on import or set low_memory=False.
  Genero = pd.read_csv('/content/ActorVial.csv')


In [ ]:
# 2. Revisión de la Estructura del Dataset
print('El DataFrame está compuesto por {:,}'.format(Genero.shape[0]),'filas y',Genero.shape[1],'columnas')

El DataFrame está compuesto por 1,676,598 filas y 1 columnas


In [ ]:
# 3. Eliminamos los duplicados
Genero = Genero.drop_duplicates()

In [ ]:
# 4. Eliminamos los valores vacíos
Genero = Genero.dropna()

In [ ]:
# 5. Reseteamos el ínidce y confimamos  de la Estructura del Dataset
Genero = Genero.reset_index(drop=True)
print('El DataFrame está compuesto por {:,}'.format(Genero.shape[0]),'filas y',Genero.shape[1],'columnas')

El DataFrame está compuesto por 3 filas y 1 columnas


In [ ]:
#6. Creamos codigo de genero
Genero['COD_GENERO']=Genero.index+1

In [ ]:
print(Genero.head())

            GENERO  COD_GENERO
0        MASCULINO           1
1         FEMENINO           2
2  SIN INFORMACION           3


In [ ]:
Genero["COD_GENERO"] = pd.to_numeric(Genero["COD_GENERO"] , downcast="float", errors="coerce")
Genero["COD_GENERO"] = Genero["COD_GENERO"].astype(float)

In [ ]:
print('Las variables almacenan la información en el siguiente formato:', Genero.dtypes)

Las variables almacenan la información en el siguiente formato: GENERO         object
COD_GENERO    float64
dtype: object


### 3.27 Creación tabla Actor Vial

In [ ]:
Actor_Vial = pd.read_csv('/content/ActorVial.csv')
Actor_Vial.head()

<ipython-input-266-bc8cf83b652a>:1: DtypeWarning: Columns (1,8) have mixed types. Specify dtype option on import or set low_memory=False.
  Actor_Vial = pd.read_csv('/content/ActorVial.csv')


,OBJECTID,FORMULARIO,CODIGO_ACCIDENTADO,CODIGO_VICTIMA,CODIGO_VEHICULO,CONDICION,ESTADO,MUERTE_POSTERIOR,FECHA_POSTERIOR_MUERTE,GENERO,FECHA_NACIMIENTO,EDAD,CODIGO,CONDICION_VEHICULO
0,1,A001234819,12725004.0,0,1.0,CONDUCTOR,ILESO,NaN,NaN,MASCULINO,1991/04/28 00:00:00+00,29.0,10528705-1,CONDUCTOR
1,2,A001234819,12725005.0,0,2.0,CONDUCTOR,ILESO,NaN,NaN,MASCULINO,1956/04/01 00:00:01+00,64.0,10528705-2,CONDUCTOR
2,3,A001234906,12725006.0,0,1.0,CONDUCTOR,ILESO,NaN,NaN,MASCULINO,1997/06/23 00:00:00+00,23.0,10528709-1,CONDUCTOR
3,4,A001234906,12725007.0,0,2.0,CONDUCTOR,ILESO,NaN,NaN,FEMENINO,1985/05/25 00:00:00+00,35.0,10528709-2,CONDUCTOR
4,5,A001232695,12725008.0,0,1.0,CONDUCTOR,ILESO,NaN,NaN,MASCULINO,1979/10/04 00:00:00+00,41.0,10528708-1,CONDUCTOR


In [ ]:
# 1. Revisión de la Estructura del Dataset
print('El DataFrame está compuesto por {:,}'.format(Actor_Vial.shape[0]),'filas y',Actor_Vial.shape[1],'columnas')
print('Con información distribuida en las siguientes variables:', Actor_Vial.columns)
print('Las variables almacenan la información en el siguiente formato:', Actor_Vial.dtypes)

El DataFrame está compuesto por 1,676,598 filas y 14 columnas
Con información distribuida en las siguientes variables: Index(['OBJECTID', 'FORMULARIO', 'CODIGO_ACCIDENTADO', 'CODIGO_VICTIMA',
       'CODIGO_VEHICULO', 'CONDICION', 'ESTADO', 'MUERTE_POSTERIOR',
       'FECHA_POSTERIOR_MUERTE', 'GENERO', 'FECHA_NACIMIENTO', 'EDAD',
       'CODIGO', 'CONDICION_VEHICULO'],
      dtype='object')
Las variables almacenan la información en el siguiente formato: OBJECTID                    int64
FORMULARIO                 object
CODIGO_ACCIDENTADO        float64
CODIGO_VICTIMA              int64
CODIGO_VEHICULO           float64
CONDICION                  object
ESTADO                     object
MUERTE_POSTERIOR           object
FECHA_POSTERIOR_MUERTE     object
GENERO                     object
FECHA_NACIMIENTO           object
EDAD                      float64
CODIGO                     object
CONDICION_VEHICULO         object
dtype: object


In [ ]:
# 2. Exploración de los Datos
description = Actor_Vial.describe(include='all')
description

,OBJECTID,FORMULARIO,CODIGO_ACCIDENTADO,CODIGO_VICTIMA,CODIGO_VEHICULO,CONDICION,ESTADO,MUERTE_POSTERIOR,FECHA_POSTERIOR_MUERTE,GENERO,FECHA_NACIMIENTO,EDAD,CODIGO,CONDICION_VEHICULO
count,1.676598e+06,1676598,1.676595e+06,1.676598e+06,1.557445e+06,1676569,1670941,496386,4149,1672288,1676309,1.514637e+06,1676598,1676569
unique,NaN,811646,NaN,NaN,NaN,6,3,2,2989,3,80270,NaN,1549926,7
top,NaN,A000239725,NaN,NaN,NaN,CONDUCTOR,ILESO,N,2006/12/16 00:00:00+00,MASCULINO,1900/01/01 00:00:01+00,NaN,10503205-3,CONDUCTOR
freq,NaN,75,NaN,NaN,NaN,1231628,1257506,492245,8,1358171,162247,NaN,59,1231628
mean,8.382995e+05,NaN,3.705871e+06,2.859696e-01,1.493111e+00,NaN,NaN,NaN,NaN,NaN,NaN,3.667145e+01,NaN,NaN
std,4.839923e+05,NaN,4.562760e+06,1.222645e+00,6.376130e-01,NaN,NaN,NaN,NaN,NaN,NaN,3.959473e+01,NaN,NaN
min,1.000000e+00,NaN,1.648400e+04,0.000000e+00,-1.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,-7.624000e+03,NaN,NaN
25%,4.191502e+05,NaN,6.333675e+05,0.000000e+00,1.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,2.700000e+01,NaN,NaN
50%,8.382995e+05,NaN,2.188181e+06,0.000000e+00,1.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,3.500000e+01,NaN,NaN
75%,1.257449e+06,NaN,2.612886e+06,0.000000e+00,2.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,4.600000e+01,NaN,NaN


In [ ]:
# 3. Traemos los codigos de condicion_actor, estado y genero
Actor_Vial = pd.merge(Actor_Vial,Condicion_Actor, how="left", left_on="CONDICION", right_on="CONDICION")
Actor_Vial = pd.merge(Actor_Vial,Estado_ActorVial, how="left", left_on="ESTADO", right_on="ESTADO")
Actor_Vial = pd.merge(Actor_Vial,Genero, how="left", left_on="GENERO", right_on="GENERO")

In [ ]:
# 4. Creamos columna Ano_Nacimiento
Actor_Vial['Ano_Nacimiento'] = ""

In [ ]:
# 5. Transformamos fecha de nacimiento a texto y extraemos sus primeros 4 caracteres
Actor_Vial['FECHA_NACIMIENTO'] = Actor_Vial['FECHA_NACIMIENTO'].astype(str)
Actor_Vial["Ano_Nacimiento"] = Actor_Vial['FECHA_NACIMIENTO'].apply(lambda x: x[:4])

In [ ]:
# 6. Convertimos el año a número entero
Actor_Vial["Ano_Nacimiento"] = pd.to_numeric(Actor_Vial["Ano_Nacimiento"], downcast="integer", errors="coerce")

In [ ]:
# 7. Reemplazamos todas las fechas de 1900 por NaN
Actor_Vial['FECHA_NACIMIENTO']=Actor_Vial['FECHA_NACIMIENTO'].replace(['1900/01/01 00:00:00+00'],[np.nan])

In [ ]:
# 8. Reemplazamos todas las fechas incongruentes por NaN
fecha_actual = datetime.date.today()
ano_actual = fecha_actual.year
for i in range(len(Actor_Vial["OBJECTID"])):
  if Actor_Vial.loc[i,"Ano_Nacimiento"] > ano_actual:
    Actor_Vial.loc[i, "FECHA_NACIMIENTO"] = np.nan

In [ ]:
# 9. Cambio formato de campo FECHA_NACIMIENTO
Actor_Vial.loc[:, 'FECHA_NACIMIENTO'] = pd.to_datetime(Actor_Vial['FECHA_NACIMIENTO']).dt.strftime('%d/%m/%Y')
Actor_Vial["FECHA_NACIMIENTO"]=pd.to_datetime(Actor_Vial["FECHA_NACIMIENTO"], format="%d/%m/%Y")

In [ ]:
# 10. Crear Columna Codigo Accidente
Actor_Vial["CODIGO_ACCIDENTE"] = ""
Actor_Vial["CODIGO_ACCIDENTE"] = Actor_Vial['CODIGO'].str.split('-').str[0]

In [ ]:
# 11. Aseguramos el tipo de datos que almacenará la columna
Actor_Vial["CODIGO_ACCIDENTADO"].fillna(value=0, inplace=True)
Actor_Vial["CODIGO_ACCIDENTADO"] = pd.to_numeric(Actor_Vial["CODIGO_ACCIDENTADO"] , downcast="integer", errors="coerce")
Actor_Vial['CODIGO_ACCIDENTADO']=Actor_Vial['CODIGO_ACCIDENTADO'].replace([0],[np.nan])
Actor_Vial["CODIGO_ACCIDENTE"] = pd.to_numeric(Actor_Vial["CODIGO_ACCIDENTE"] , downcast="integer", errors="coerce")
Actor_Vial["CODIGO_ACCIDENTE"] = Actor_Vial["CODIGO_ACCIDENTE"].astype('int64')

In [ ]:
# 12. Contabilizamos valores nulos, e identificamos participación en el total de registros por columnas
resumen_NaN_ActorVial = pd.DataFrame(Actor_Vial.isnull().sum())
resumen_NaN_ActorVial.columns=['Nulos']
resumen_NaN_ActorVial['Registros']=Actor_Vial.shape[0]
resumen_NaN_ActorVial['% Nulos']=resumen_NaN_ActorVial.Nulos/resumen_NaN_ActorVial.Registros
resumen_NaN_ActorVial['% Nulos']=resumen_NaN_ActorVial['% Nulos'].apply("{:.2%}".format)
resumen_NaN_ActorVial

,Nulos,Registros,% Nulos
OBJECTID,0,1676598,0.00%
FORMULARIO,0,1676598,0.00%
CODIGO_ACCIDENTADO,3,1676598,0.00%
CODIGO_VICTIMA,0,1676598,0.00%
CODIGO_VEHICULO,119153,1676598,7.11%
CONDICION,29,1676598,0.00%
ESTADO,5657,1676598,0.34%
MUERTE_POSTERIOR,1180212,1676598,70.39%
FECHA_POSTERIOR_MUERTE,1672449,1676598,99.75%
GENERO,4310,1676598,0.26%


In [ ]:
# 13. Traemos los codigos de accidente para conservar solo los registros que coinciden con códigos de accidente
Actor_Vial = pd.merge(Actor_Vial, Accidente_df, how='left', left_on='CODIGO_ACCIDENTE',
                         right_on='CODIGO_ACCIDENTE')

In [ ]:
# 14. Contabilizamos valores nulos, e identificamos participación en el total de registros por columnas
resumen_NaN_Actor_Vial2 = pd.DataFrame(Actor_Vial.isnull().sum())
resumen_NaN_Actor_Vial2.columns=['Nulos']
resumen_NaN_Actor_Vial2['Registros']=Actor_Vial.shape[0]
resumen_NaN_Actor_Vial2['% Nulos']=resumen_NaN_Actor_Vial2.Nulos/resumen_NaN_Actor_Vial2.Registros
resumen_NaN_Actor_Vial2['% Nulos']=resumen_NaN_Actor_Vial2['% Nulos'].apply("{:.2%}".format)
resumen_NaN_Actor_Vial2

,Nulos,Registros,% Nulos
OBJECTID,0,1676598,0.00%
FORMULARIO,0,1676598,0.00%
CODIGO_ACCIDENTADO,3,1676598,0.00%
CODIGO_VICTIMA,0,1676598,0.00%
CODIGO_VEHICULO,119153,1676598,7.11%
CONDICION,29,1676598,0.00%
ESTADO,5657,1676598,0.34%
MUERTE_POSTERIOR,1180212,1676598,70.39%
FECHA_POSTERIOR_MUERTE,1672449,1676598,99.75%
GENERO,4310,1676598,0.26%


In [ ]:
# 15. Se eliminan todas las filas en las que la fecha de accidente está vacía,
# pues indica que es un codigo de accidente inexistente en la tabla accidente
Actor_Vial = Actor_Vial.dropna(subset=['FECHA_OCURRENCIA_ACC'])

In [ ]:
# 16. Crear un nuevo DataFrame con las columnas seleccionadas
selected_columns = [
    'CODIGO_ACCIDENTADO',
    'COD_CONDICION_ACTOR',
    'COD_ESTADO',
    'COD_GENERO',
    'FECHA_NACIMIENTO',
    'CODIGO',
    'CODIGO_ACCIDENTE'
]

Actor_Vial_df= Actor_Vial[selected_columns]
Actor_Vial_df = pd.DataFrame(Actor_Vial_df)


In [ ]:
# 17. Identificar registros duplicados
duplicates = Actor_Vial_df.duplicated()

# Contar la cantidad de registros duplicados
num_duplicates = sum(duplicates)

print("Número de registros duplicados:", num_duplicates)
print("Registros duplicados:")
print(Actor_Vial_df[duplicates])

Número de registros duplicados: 0
Registros duplicados:
Empty DataFrame
Columns: [CODIGO_ACCIDENTADO, COD_CONDICION_ACTOR, COD_ESTADO, COD_GENERO, FECHA_NACIMIENTO, CODIGO, CODIGO_ACCIDENTE]
Index: []


In [ ]:
# 18. Visualizar el nuevo DataFrame con las columnas seleccionadas
print(Actor_Vial_df.head())

   CODIGO_ACCIDENTADO  COD_CONDICION_ACTOR  COD_ESTADO  COD_GENERO  \
0          12725004.0                  1.0         1.0         1.0   
1          12725005.0                  1.0         1.0         1.0   
2          12725006.0                  1.0         1.0         1.0   
3          12725007.0                  1.0         1.0         2.0   
4          12725008.0                  1.0         1.0         1.0   

  FECHA_NACIMIENTO      CODIGO  CODIGO_ACCIDENTE  
0       1991-04-28  10528705-1          10528705  
1       1956-04-01  10528705-2          10528705  
2       1997-06-23  10528709-1          10528709  
3       1985-05-25  10528709-2          10528709  
4       1979-10-04  10528708-1          10528708  


In [ ]:
# 19. Almacenamos solo la fecha de fecha nacimiento y muerte posterior
Actor_Vial_df['FECHA_NACIMIENTO'] = Actor_Vial_df['FECHA_NACIMIENTO'].dt.date

In [ ]:
print('Las variables almacenan la información en el siguiente formato:', Actor_Vial_df.dtypes)

Las variables almacenan la información en el siguiente formato: CODIGO_ACCIDENTADO     float64
COD_CONDICION_ACTOR    float64
COD_ESTADO             float64
COD_GENERO             float64
FECHA_NACIMIENTO        object
CODIGO                  object
CODIGO_ACCIDENTE         int64
dtype: object


### 3.28  Creación tabla Via

In [ ]:
Via = pd.read_csv('/content/Via.csv')

<ipython-input-287-6cd40d78cfbd>:1: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  Via = pd.read_csv('/content/Via.csv')


In [ ]:
Via.tail(5)

,OBJECTID,FORMULARIO,CODIGO_ACCIDENTE,CODIGO_VIA,GEOMETRICA_A,GEOMETRICA_B,GEOMETRICA_C,UTILIZACION,CALZADAS,CARRILES,MATERIAL,ESTADO,CONDICIONES,ILUMINACION_A,ILUMINACION_B,AGENTE_TRANSITO,SEMAFORO,VISUAL,CODIGO
483947,11391602,A001232983,10530494,1,RECTA,PLANO,CON BERMAS,UN SENTIDO,UNA,TRES O MAS,ASFALTO,BUENO,HUMEDA,SIN,NaN,NO,OPERANDO,NaN,11391602-10530494-1
483948,11391603,A001236189,10530495,1,RECTA,PLANO,CON ANDEN,UN SENTIDO,UNA,DOS,ASFALTO,BUENO,SECA,SIN,NaN,NO,OPERANDO,NaN,11391603-10530495-1
483949,11391604,A001236189,10530495,2,RECTA,PLANO,CON ANDEN,DOBLE SENTIDO,UNA,DOS,ASFALTO,BUENO,SECA,SIN,NaN,NO,OPERANDO,NaN,11391604-10530495-2
483950,11391605,A001234696,10530496,1,RECTA,PLANO,CON ANDEN,DOBLE SENTIDO,DOS,TRES O MAS,ASFALTO,BUENO,HUMEDA,CON,BUENA,NO,OPERANDO,NaN,11391605-10530496-1
483951,11391607,A001236124,10530498,1,RECTA,PLANO,CON ANDEN,DOBLE SENTIDO,UNA,DOS,ASFALTO,BUENO,SECA,SIN,NaN,NO,OPERANDO,NaN,11391607-10530498-1


In [ ]:
# 1. Revisión de la Estructura del Dataset
print('El DataFrame está compuesto por {:,}'.format(Via.shape[0]),'filas y',Via.shape[1],'columnas')
print('Con información distribuida en las siguientes variables:', Via.columns)
print('Las variables almacenan la información en el siguiente formato:', Via.dtypes)

El DataFrame está compuesto por 483,952 filas y 19 columnas
Con información distribuida en las siguientes variables: Index(['OBJECTID', 'FORMULARIO', 'CODIGO_ACCIDENTE', 'CODIGO_VIA',
       'GEOMETRICA_A', 'GEOMETRICA_B', 'GEOMETRICA_C', 'UTILIZACION',
       'CALZADAS', 'CARRILES', 'MATERIAL', 'ESTADO', 'CONDICIONES',
       'ILUMINACION_A', 'ILUMINACION_B', 'AGENTE_TRANSITO', 'SEMAFORO',
       'VISUAL', 'CODIGO'],
      dtype='object')
Las variables almacenan la información en el siguiente formato: OBJECTID             int64
FORMULARIO          object
CODIGO_ACCIDENTE     int64
CODIGO_VIA           int64
GEOMETRICA_A        object
GEOMETRICA_B        object
GEOMETRICA_C        object
UTILIZACION         object
CALZADAS            object
CARRILES            object
MATERIAL            object
ESTADO              object
CONDICIONES         object
ILUMINACION_A       object
ILUMINACION_B       object
AGENTE_TRANSITO     object
SEMAFORO            object
VISUAL              object
CODIGO

In [ ]:
# 2. Exploración de los Datos
description = Via.describe(include='all')
description

,OBJECTID,FORMULARIO,CODIGO_ACCIDENTE,CODIGO_VIA,GEOMETRICA_A,GEOMETRICA_B,GEOMETRICA_C,UTILIZACION,CALZADAS,CARRILES,MATERIAL,ESTADO,CONDICIONES,ILUMINACION_A,ILUMINACION_B,AGENTE_TRANSITO,SEMAFORO,VISUAL,CODIGO
count,4.839520e+05,483952,4.839520e+05,483952.000000,482328,477523,473775,480598,479607,480908,480233,479239,481109,463782,205161,440178,284638,717,483952
unique,NaN,392620,NaN,NaN,2,2,3,5,5,5,7,9,8,2,2,2,6,6,483952
top,NaN,A000758222,NaN,NaN,RECTA,PLANO,CON ANDEN,UN SENTIDO,UNA,DOS,ASFALTO,BUENO,SECA,SIN,BUENA,NO,OPERANDO,VEHICULO ESTACIONADO,526-511950-1
freq,NaN,102,NaN,NaN,466677,456910,335800,285020,342679,241070,432202,461780,438656,259884,202335,434599,234897,430,1
mean,4.752536e+06,NaN,3.331932e+06,1.182152,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,3.655194e+06,NaN,3.880704e+06,0.385970,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,5.260000e+02,NaN,1.040000e+02,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,2.052829e+06,NaN,3.704858e+05,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,4.489721e+06,NaN,4.888085e+05,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,5.343975e+06,NaN,4.489740e+06,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# 3. Contabilizamos valores nulos, e identificamos participación en el total de registros por columnas
resumen_NaN_Via = pd.DataFrame(Via.isnull().sum())
resumen_NaN_Via.columns=['Nulos']
resumen_NaN_Via['Registros']=Via.shape[0]
resumen_NaN_Via['% Nulos']=resumen_NaN_Via.Nulos/resumen_NaN_Via.Registros
resumen_NaN_Via['% Nulos']=resumen_NaN_Via['% Nulos'].apply("{:.2%}".format)
resumen_NaN_Via

,Nulos,Registros,% Nulos
OBJECTID,0,483952,0.00%
FORMULARIO,0,483952,0.00%
CODIGO_ACCIDENTE,0,483952,0.00%
CODIGO_VIA,0,483952,0.00%
GEOMETRICA_A,1624,483952,0.34%
GEOMETRICA_B,6429,483952,1.33%
GEOMETRICA_C,10177,483952,2.10%
UTILIZACION,3354,483952,0.69%
CALZADAS,4345,483952,0.90%
CARRILES,3044,483952,0.63%


Se podría presumir la eliminación de las varibles visual e Iluminación B si para el 99,9% y 57,6%, respectivamente, de sus registros no está aportando información adicional

In [ ]:
# 4. Traemos los codigos de geometrica A-B-C, utilizacion, carriles,material, estado, condiciones, iluminacionA,
# agente_transito y semaforo que corresponde al indice de su dimensión
Via = pd.merge(Via,Geometrica_A, how="left", left_on="GEOMETRICA_A", right_on="GEOMETRICA_A")
Via = pd.merge(Via,Geometrica_B, how="left", left_on="GEOMETRICA_B", right_on="GEOMETRICA_B")
Via = pd.merge(Via,Geometrica_C, how="left", left_on="GEOMETRICA_C", right_on="GEOMETRICA_C")
Via = pd.merge(Via,Utilizacion, how="left", left_on="UTILIZACION", right_on="UTILIZACION")
Via = pd.merge(Via,Calzadas, how="left", left_on="CALZADAS", right_on="CALZADAS")
Via = pd.merge(Via,Carriles, how="left", left_on="CARRILES", right_on="CARRILES")
Via = pd.merge(Via,Material, how="left", left_on="MATERIAL", right_on="MATERIAL")
Via = pd.merge(Via,Estado_Via, how="left", left_on="ESTADO", right_on="ESTADO")
Via = pd.merge(Via,Condiciones, how="left", left_on="CONDICIONES", right_on="CONDICIONES")
Via = pd.merge(Via,Iluminacion_A, how="left", left_on="ILUMINACION_A", right_on="ILUMINACION_A")
Via = pd.merge(Via,Agente_Transito, how="left", left_on="AGENTE_TRANSITO", right_on="AGENTE_TRANSITO")
Via = pd.merge(Via,Semaforo, how="left", left_on="SEMAFORO", right_on="SEMAFORO")

In [ ]:
# 5. Traemos los codigos de accidente para conservar solo los registros que coinciden con códigos de accidente
Via = pd.merge(Via, Accidente_df, how='left', left_on='CODIGO_ACCIDENTE',
                         right_on='CODIGO_ACCIDENTE')

In [ ]:
# 6. Contabilizamos valores nulos, e identificamos participación en el total de registros por columnas
resumen_NaN_Via2 = pd.DataFrame(Via.isnull().sum())
resumen_NaN_Via2.columns=['Nulos']
resumen_NaN_Via2['Registros']=Via.shape[0]
resumen_NaN_Via2['% Nulos']=resumen_NaN_Via2.Nulos/resumen_NaN_Via2.Registros
resumen_NaN_Via2['% Nulos']=resumen_NaN_Via2['% Nulos'].apply("{:.2%}".format)
resumen_NaN_Via2


,Nulos,Registros,% Nulos
OBJECTID,0,483952,0.00%
FORMULARIO,0,483952,0.00%
CODIGO_ACCIDENTE,0,483952,0.00%
CODIGO_VIA,0,483952,0.00%
GEOMETRICA_A,1624,483952,0.34%
GEOMETRICA_B,6429,483952,1.33%
GEOMETRICA_C,10177,483952,2.10%
UTILIZACION,3354,483952,0.69%
CALZADAS,4345,483952,0.90%
CARRILES,3044,483952,0.63%


In [ ]:
# 7. Se eliminan todas las filas en las que la fecha de accidente está vacía,
# pues indica que es un codigo de accidente inexistente en la tabla accidente
Via = Via.dropna(subset=['FECHA_OCURRENCIA_ACC'])

In [ ]:
# 8. Seleccionar solo las columnas deseadas
selected_columns = [
    'CODIGO_ACCIDENTE', 'COD_GEOMETRICA_A', 'COD_GEOMETRICA_B', 'COD_GEOMETRICA_C',
    'COD_UTILIZACION', 'COD_CALZADAS', 'COD_CARRILES', 'COD_MATERIAL', 'COD_ESTADO', 'COD_CONDICION',
    'COD_ILUMINACION_A', 'COD_AGENTE_TRANSITO', 'COD_SEMAFORO', 'CODIGO'
]
Via_df = Via[selected_columns]
Via_df = pd.DataFrame(Via_df)

In [ ]:
# 9. Identificar registros duplicados
duplicates = Via_df.duplicated()

# 10. Contar la cantidad de registros duplicados
num_duplicates = sum(duplicates)

print("Número de registros duplicados:", num_duplicates)
print(Via_df[duplicates])

Número de registros duplicados: 0
Empty DataFrame
Columns: [CODIGO_ACCIDENTE, COD_GEOMETRICA_A, COD_GEOMETRICA_B, COD_GEOMETRICA_C, COD_UTILIZACION, COD_CALZADAS, COD_CARRILES, COD_MATERIAL, COD_ESTADO, COD_CONDICION, COD_ILUMINACION_A, COD_AGENTE_TRANSITO, COD_SEMAFORO, CODIGO]
Index: []


In [ ]:
# 11. Asegurar tipo de datos
Via_df["CODIGO_ACCIDENTE"] = pd.to_numeric(Via_df["CODIGO_ACCIDENTE"], downcast="integer", errors="coerce")
Via_df["CODIGO_ACCIDENTE"] = Via_df["CODIGO_ACCIDENTE"].astype('int64')
Via_df["CODIGO_ACCIDENTE"] = round(Via_df["CODIGO_ACCIDENTE"].astype(int),0)
Via_df["COD_GEOMETRICA_A"] = round(pd.to_numeric(Via_df["COD_GEOMETRICA_A"], downcast="integer", errors="coerce"),0)
Via_df["COD_GEOMETRICA_B"] = pd.to_numeric(Via_df["COD_GEOMETRICA_B"], downcast="integer", errors="coerce")
Via_df["COD_GEOMETRICA_C"] = pd.to_numeric(Via_df["COD_GEOMETRICA_C"], downcast="integer", errors="coerce")
Via_df["COD_UTILIZACION"] = pd.to_numeric(Via_df["COD_UTILIZACION"], downcast="integer", errors="coerce")
Via_df["COD_CALZADAS"] = pd.to_numeric(Via_df["COD_CALZADAS"], downcast="integer", errors="coerce")
Via_df["COD_CARRILES"] = pd.to_numeric(Via_df["COD_CARRILES"], downcast="integer", errors="coerce")
Via_df["COD_MATERIAL"] = pd.to_numeric(Via_df["COD_MATERIAL"], downcast="integer", errors="coerce")
Via_df["COD_ESTADO"] = pd.to_numeric(Via_df["COD_ESTADO"], downcast="integer", errors="coerce")
Via_df["COD_CONDICION"] = pd.to_numeric(Via_df["COD_CONDICION"], downcast="integer", errors="coerce")
Via_df["COD_ILUMINACION_A"] = pd.to_numeric(Via_df["COD_ILUMINACION_A"], downcast="integer", errors="coerce")
Via_df["COD_AGENTE_TRANSITO"] = pd.to_numeric(Via_df["COD_AGENTE_TRANSITO"], downcast="integer", errors="coerce")
Via_df["COD_SEMAFORO"] = pd.to_numeric(Via_df["COD_SEMAFORO"], downcast="integer", errors="coerce")

In [ ]:
# 12. Visualizar el tipo de las columnas seleccionadas
print('Las variables almacenan la información en el siguiente formato:', Via_df.dtypes)

Las variables almacenan la información en el siguiente formato: CODIGO_ACCIDENTE         int64
COD_GEOMETRICA_A       float64
COD_GEOMETRICA_B       float64
COD_GEOMETRICA_C       float64
COD_UTILIZACION        float64
COD_CALZADAS           float64
COD_CARRILES           float64
COD_MATERIAL           float64
COD_ESTADO             float64
COD_CONDICION          float64
COD_ILUMINACION_A      float64
COD_AGENTE_TRANSITO    float64
COD_SEMAFORO           float64
CODIGO                  object
dtype: object


### 3.29 Creación tabla Causa

In [ ]:
Causa = pd.read_csv('/content/Causa.csv')
Causa.head()

<ipython-input-300-96e993aba28a>:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  Causa = pd.read_csv('/content/Causa.csv')


,OBJECTID,FORMULARIO,CODIGO_ACCIDENTE,CODIGO_VEHICULO,CODIGO_CAUSA,NOMBRE,TIPO,TIPO_CAUSA,CODIGO,CODIGO_AC_VH
0,1,A001387863,10558992,1,157,OTRA,CG,CONDUCTOR,10558992-1-157,10558992-1
1,2,A001387863,10558992,2,157,OTRA,CG,CONDUCTOR,10558992-2-157,10558992-2
2,3,A001388525,10558993,2,121,NO MANTENER DISTANCIA DE SEGURIDAD,CG,CONDUCTOR,10558993-2-121,10558993-2
3,4,A001388525,10558993,2,157,OTRA,CG,CONDUCTOR,10558993-2-157,10558993-2
4,5,A001387465,10558994,1,143,PONER EN MARCHA UN VEHICULO SIN PRECAUCIONES,CG,CONDUCTOR,10558994-1-143,10558994-1


In [ ]:
# 1. Revisión de la Estructura del Dataset
print('El DataFrame está compuesto por {:,}'.format(Causa.shape[0]),'filas y',Causa.shape[1],'columnas')
print('Con información distribuida en las siguientes variables:', Causa.columns)
print('Las variables almacenan la información en el siguiente formato:', Causa.dtypes)

El DataFrame está compuesto por 682,392 filas y 10 columnas
Con información distribuida en las siguientes variables: Index(['OBJECTID', 'FORMULARIO', 'CODIGO_ACCIDENTE', 'CODIGO_VEHICULO',
       'CODIGO_CAUSA', 'NOMBRE', 'TIPO', 'TIPO_CAUSA', 'CODIGO',
       'CODIGO_AC_VH'],
      dtype='object')
Las variables almacenan la información en el siguiente formato: OBJECTID             int64
FORMULARIO          object
CODIGO_ACCIDENTE     int64
CODIGO_VEHICULO      int64
CODIGO_CAUSA        object
NOMBRE              object
TIPO                object
TIPO_CAUSA          object
CODIGO              object
CODIGO_AC_VH        object
dtype: object


In [ ]:
# 2. Exploración de los Datos
description = Causa.describe(include='all')
description

,OBJECTID,FORMULARIO,CODIGO_ACCIDENTE,CODIGO_VEHICULO,CODIGO_CAUSA,NOMBRE,TIPO,TIPO_CAUSA,CODIGO,CODIGO_AC_VH
count,682392.000000,682392,6.823920e+05,682392.00000,682392,680746,677059,677059,682392,682392
unique,NaN,468112,NaN,NaN,234,119,6,5,682212,672043
top,NaN,A00728644,NaN,NaN,157,OTRA,CG,CONDUCTOR,10543594-2-121,10543594-2
freq,NaN,11,NaN,NaN,223865,244798,614393,623368,6,6
mean,341196.500000,NaN,3.696610e+06,1.37990,NaN,NaN,NaN,NaN,NaN,NaN
std,196989.746784,NaN,4.278107e+06,0.60393,NaN,NaN,NaN,NaN,NaN,NaN
min,1.000000,NaN,1.040000e+02,0.00000,NaN,NaN,NaN,NaN,NaN,NaN
25%,170598.750000,NaN,3.818358e+05,1.00000,NaN,NaN,NaN,NaN,NaN,NaN
50%,341196.500000,NaN,4.865860e+05,1.00000,NaN,NaN,NaN,NaN,NaN,NaN
75%,511794.250000,NaN,1.045033e+07,2.00000,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# 3. Contabilizamos valores nulos, e identificamos participación en el total de registros por columnas
resumen_NaN_Causa = pd.DataFrame(Causa.isnull().sum())
resumen_NaN_Causa.columns=['Nulos']
resumen_NaN_Causa['Registros']=Causa.shape[0]
resumen_NaN_Causa['% Nulos']=resumen_NaN_Causa.Nulos/resumen_NaN_Causa.Registros
resumen_NaN_Causa['% Nulos']=resumen_NaN_Causa['% Nulos'].apply("{:.2%}".format)
resumen_NaN_Causa

,Nulos,Registros,% Nulos
OBJECTID,0,682392,0.00%
FORMULARIO,0,682392,0.00%
CODIGO_ACCIDENTE,0,682392,0.00%
CODIGO_VEHICULO,0,682392,0.00%
CODIGO_CAUSA,0,682392,0.00%
NOMBRE,1646,682392,0.24%
TIPO,5333,682392,0.78%
TIPO_CAUSA,5333,682392,0.78%
CODIGO,0,682392,0.00%
CODIGO_AC_VH,0,682392,0.00%


In [ ]:
# 4. Identificar registros duplicados
duplicates = Causa.duplicated()

# Contar la cantidad de registros duplicados
num_duplicates = sum(duplicates)

print("Número de registros duplicados:", num_duplicates)
print(Causa[duplicates])


Número de registros duplicados: 0
Empty DataFrame
Columns: [OBJECTID, FORMULARIO, CODIGO_ACCIDENTE, CODIGO_VEHICULO, CODIGO_CAUSA, NOMBRE, TIPO, TIPO_CAUSA, CODIGO, CODIGO_AC_VH]
Index: []


In [ ]:
# 5. Traemos los codigos de descripción y tipo de causa
Causa["CODIGO_CAUSA"] = pd.to_numeric(Causa["CODIGO_CAUSA"] , downcast="float", errors="coerce")
Causa["CODIGO_CAUSA"] = Causa["CODIGO_CAUSA"].astype(float)
Causa["CODIGO_ACCIDENTE"] = pd.to_numeric(Causa["CODIGO_ACCIDENTE"] , downcast="integer", errors="coerce")
Causa["CODIGO_ACCIDENTE"] = Causa["CODIGO_ACCIDENTE"].astype('int64')
Causa = pd.merge(Causa,Describe_Causa, how="left", left_on="CODIGO_CAUSA", right_on="CODIGO_CAUSA")
Causa = pd.merge(Causa,Tipo_Causa, how="left", left_on="TIPO", right_on="TIPO")

In [ ]:
# 6. Traemos los codigos de accidente para conservar solo los registros que coinciden con códigos de accidente
Causa = pd.merge(Causa, Accidente_df, how='left', left_on='CODIGO_ACCIDENTE',
                         right_on='CODIGO_ACCIDENTE')

In [ ]:
# 7. Contabilizamos valores nulos, e identificamos participación en el total de registros por columnas
resumen_NaN_Causa2 = pd.DataFrame(Causa.isnull().sum())
resumen_NaN_Causa2.columns=['Nulos']
resumen_NaN_Causa2['Registros']=Causa.shape[0]
resumen_NaN_Causa2['% Nulos']=resumen_NaN_Causa2.Nulos/resumen_NaN_Causa2.Registros
resumen_NaN_Causa2['% Nulos']=resumen_NaN_Causa2['% Nulos'].apply("{:.2%}".format)
resumen_NaN_Causa2

,Nulos,Registros,% Nulos
OBJECTID,0,682392,0.00%
FORMULARIO,0,682392,0.00%
CODIGO_ACCIDENTE,0,682392,0.00%
CODIGO_VEHICULO,0,682392,0.00%
CODIGO_CAUSA,9765,682392,1.43%
NOMBRE_x,1646,682392,0.24%
TIPO,5333,682392,0.78%
TIPO_CAUSA_x,5333,682392,0.78%
CODIGO,0,682392,0.00%
CODIGO_AC_VH,0,682392,0.00%


In [ ]:
# 8. Se eliminan todas las filas en las que la fecha de accidente está vacía,
# pues indica que es un codigo de accidente inexistente en la tabla accidente
Causa = Causa.dropna(subset=['FECHA_OCURRENCIA_ACC'])

In [ ]:
# 9. Crear un nuevo DataFrame con las columnas seleccionadas
selected_columns =[
    'CODIGO_ACCIDENTE','CODIGO_CAUSA', 'TIPO', 'CODIGO_AC_VH'
]

Causa_df= Causa[selected_columns]
Causa_df = pd.DataFrame(Causa_df)


In [ ]:
Causa_df.head()

,CODIGO_ACCIDENTE,CODIGO_CAUSA,TIPO,CODIGO_AC_VH
0,10558992,157.0,CG,10558992-1
1,10558992,157.0,CG,10558992-2
2,10558993,121.0,CG,10558993-2
3,10558993,157.0,CG,10558993-2
4,10558994,143.0,CG,10558994-1


In [ ]:
# 10. Contabilizamos valores nulos, e identificamos participación en el total de registros por columnas
resumen_NaN_Causa_df = pd.DataFrame(Causa_df.isnull().sum())
resumen_NaN_Causa_df.columns=['Nulos']
resumen_NaN_Causa_df['Registros']=Causa_df.shape[0]
resumen_NaN_Causa_df['% Nulos']=resumen_NaN_Causa_df.Nulos/resumen_NaN_Causa_df.Registros
resumen_NaN_Causa_df['% Nulos']=resumen_NaN_Causa_df['% Nulos'].apply("{:.2%}".format)
resumen_NaN_Causa_df

,Nulos,Registros,% Nulos
CODIGO_ACCIDENTE,0,682392,0.00%
CODIGO_CAUSA,9765,682392,1.43%
TIPO,5333,682392,0.78%
CODIGO_AC_VH,0,682392,0.00%


In [ ]:
print('Las variables almacenan la información en el siguiente formato:', Causa_df.dtypes)

Las variables almacenan la información en el siguiente formato: CODIGO_ACCIDENTE      int64
CODIGO_CAUSA        float64
TIPO                 object
CODIGO_AC_VH         object
dtype: object


### 3.30 Creación de tabla Vehículo

In [ ]:
Vehiculo = pd.read_csv('/content/Vehiculo.csv')
Vehiculo.head()

<ipython-input-313-a31382f0b9d1>:1: DtypeWarning: Columns (1,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  Vehiculo = pd.read_csv('/content/Vehiculo.csv')


,OBJECTID,FORMULARIO,PLACA,CODIGO_VEHICULO,CLASE,SERVICIO,MODALIDAD,ENFUGA,CODIGO
0,530,A000032685,BWH61C,1.0,MOTOCICLETA,PARTICULAR,NaN,N,511951-1
1,533,A000032986,BSU084,1.0,AUTOMOVIL,PARTICULAR,NaN,N,511950-1
2,541,A000032708,CUW805,1.0,AUTOMOVIL,PARTICULAR,NaN,N,511947-1
3,542,A000032708,YHV59C,2.0,MOTOCICLETA,PARTICULAR,NaN,N,511947-2
4,546,A000033456,WGG324,1.0,BUS,PUBLICO,PASAJEROS - MASIVO,N,511955-1


In [ ]:
# 1. Revisión de la Estructura del Dataset
print('El DataFrame está compuesto por {:,}'.format(Vehiculo.shape[0]),'filas y',Vehiculo.shape[1],'columnas')
print('Con información distribuida en las siguientes variables:', Vehiculo.columns)
print('Las variables almacenan la información en el siguiente formato:', Vehiculo.dtypes)

El DataFrame está compuesto por 1,441,024 filas y 9 columnas
Con información distribuida en las siguientes variables: Index(['OBJECTID', 'FORMULARIO', 'PLACA', 'CODIGO_VEHICULO', 'CLASE',
       'SERVICIO', 'MODALIDAD', 'ENFUGA', 'CODIGO'],
      dtype='object')
Las variables almacenan la información en el siguiente formato: OBJECTID             int64
FORMULARIO          object
PLACA               object
CODIGO_VEHICULO    float64
CLASE               object
SERVICIO            object
MODALIDAD           object
ENFUGA              object
CODIGO              object
dtype: object


In [ ]:
# 2. Exploración de los Datos
description = Vehiculo.describe(include='all')
description

,OBJECTID,FORMULARIO,PLACA,CODIGO_VEHICULO,CLASE,SERVICIO,MODALIDAD,ENFUGA,CODIGO
count,1.441024e+06,1441024.0,1393918,1.441020e+06,1427389,1405407,339198,907565,1441024
unique,NaN,780176.0,915542,NaN,27,5,12,2,1441024
top,NaN,513283600.0,P/E,NaN,AUTOMOVIL,PARTICULAR,PASAJEROS - INDIVIDUAL,N,511951-1
freq,NaN,54.0,16040,NaN,626460,772189,70712,892815,1
mean,5.461994e+06,NaN,NaN,1.502388e+00,NaN,NaN,NaN,NaN,NaN
std,5.148519e+06,NaN,NaN,6.446302e-01,NaN,NaN,NaN,NaN,NaN
min,5.300000e+02,NaN,NaN,-1.000000e+00,NaN,NaN,NaN,NaN,NaN
25%,6.676688e+05,NaN,NaN,1.000000e+00,NaN,NaN,NaN,NaN,NaN
50%,4.256345e+06,NaN,NaN,1.000000e+00,NaN,NaN,NaN,NaN,NaN
75%,8.078992e+06,NaN,NaN,2.000000e+00,NaN,NaN,NaN,NaN,NaN


In [ ]:
# 3. Contabilizamos valores nulos, e identificamos participación en el total de registros por columnas
resumen_NaN_Vehiculo = pd.DataFrame(Vehiculo.isnull().sum())
resumen_NaN_Vehiculo.columns=['Nulos']
resumen_NaN_Vehiculo['Registros']=Vehiculo.shape[0]
resumen_NaN_Vehiculo['% Nulos']=resumen_NaN_Vehiculo.Nulos/resumen_NaN_Vehiculo.Registros
resumen_NaN_Vehiculo['% Nulos']=resumen_NaN_Vehiculo['% Nulos'].apply("{:.2%}".format)
resumen_NaN_Vehiculo

,Nulos,Registros,% Nulos
OBJECTID,0,1441024,0.00%
FORMULARIO,0,1441024,0.00%
PLACA,47106,1441024,3.27%
CODIGO_VEHICULO,4,1441024,0.00%
CLASE,13635,1441024,0.95%
SERVICIO,35617,1441024,2.47%
MODALIDAD,1101826,1441024,76.46%
ENFUGA,533459,1441024,37.02%
CODIGO,0,1441024,0.00%


In [ ]:
# 4. Traemos los codigos de Clase_Vehiculo, servicio y modalidad
Vehiculo = pd.merge(Vehiculo,Clase_Vh, how="left", left_on="CLASE", right_on="CLASE")
Vehiculo = pd.merge(Vehiculo,Servicio, how="left", left_on="SERVICIO", right_on="SERVICIO")
Vehiculo = pd.merge(Vehiculo,Modalidad, how="left", left_on="MODALIDAD", right_on="MODALIDAD")

In [ ]:
# 4. Identificar registros duplicados
duplicates = Vehiculo.duplicated()

# Contar la cantidad de registros duplicados
num_duplicates = sum(duplicates)

print("Número de registros duplicados:", num_duplicates)
print(Vehiculo[duplicates])

Número de registros duplicados: 0
Empty DataFrame
Columns: [OBJECTID, FORMULARIO, PLACA, CODIGO_VEHICULO, CLASE, SERVICIO, MODALIDAD, ENFUGA, CODIGO, COD_CLASE, COD_SERVICIO, COD_MODALIDAD]
Index: []


In [ ]:
# 5. Crear Columna Codigo Accidente
Vehiculo["CODIGO_ACCIDENTE"] = Vehiculo['CODIGO'].str.split('-').str[0]

In [ ]:
# 6. Aseguramos el tipo de datos que almacenará la columna
Vehiculo["CODIGO_ACCIDENTE"] = pd.to_numeric(Vehiculo["CODIGO_ACCIDENTE"] , downcast="integer", errors="coerce")
Vehiculo["CODIGO_ACCIDENTE"] = Vehiculo["CODIGO_ACCIDENTE"].astype('int64')

In [ ]:
# 7. Traemos los codigos de accidente para conservar solo los registros que coinciden con códigos de accidente
Vehiculo = pd.merge(Vehiculo, Accidente_df, how='left', left_on='CODIGO_ACCIDENTE',
                         right_on='CODIGO_ACCIDENTE')

In [ ]:
# 8. Contabilizamos valores nulos, e identificamos participación en el total de registros por columnas
resumen_NaN_Vehiculo2 = pd.DataFrame(Vehiculo.isnull().sum())
resumen_NaN_Vehiculo2.columns=['Nulos']
resumen_NaN_Vehiculo2['Registros']=Vehiculo.shape[0]
resumen_NaN_Vehiculo2['% Nulos']=resumen_NaN_Vehiculo2.Nulos/resumen_NaN_Vehiculo2.Registros
resumen_NaN_Vehiculo2['% Nulos']=resumen_NaN_Vehiculo2['% Nulos'].apply("{:.2%}".format)
resumen_NaN_Vehiculo2

,Nulos,Registros,% Nulos
OBJECTID,0,1441024,0.00%
FORMULARIO,0,1441024,0.00%
PLACA,47106,1441024,3.27%
CODIGO_VEHICULO,4,1441024,0.00%
CLASE,13635,1441024,0.95%
SERVICIO,35617,1441024,2.47%
MODALIDAD,1101826,1441024,76.46%
ENFUGA,533459,1441024,37.02%
CODIGO,0,1441024,0.00%
COD_CLASE,13637,1441024,0.95%


In [ ]:
# 9. Se eliminan todas las filas en las que la fecha de accidente está vacía,
# pues indica que es un codigo de accidente inexistente en la tabla accidente
Vehiculo = Vehiculo.dropna(subset=['FECHA_OCURRENCIA_ACC'])

In [ ]:
# 6. Crear un nuevo DataFrame con las columnas seleccionadas
selected_columns =[
    'PLACA',
    'COD_CLASE',
    'COD_SERVICIO',
    'COD_MODALIDAD',
    'ENFUGA',
    'CODIGO',
    'CODIGO_ACCIDENTE'
]
Vehiculo_df= Vehiculo[selected_columns]
Vehiculo_df = pd.DataFrame(Vehiculo_df)


In [ ]:
print('Las variables almacenan la información en el siguiente formato:', Vehiculo_df.dtypes)

Las variables almacenan la información en el siguiente formato: PLACA                object
COD_CLASE           float64
COD_SERVICIO        float64
COD_MODALIDAD       float64
ENFUGA               object
CODIGO               object
CODIGO_ACCIDENTE      int64
dtype: object


In [ ]:
# Detiene el temporizador de transformación
transformation_end_time = time.time()

In [ ]:
# Calcula el tiempo de transformación
transformation_time = transformation_end_time - transformation_start_time
print(f'Tiempo de transformación: {transformation_time:.2f} segundos')

Tiempo de transformación: 237.40 segundos


## 4. Carga <a id="carga"></a>

In [ ]:
# Inicia el temporizador de carga
load_start_time = time.time()

### 4.1. Creación de Script de creación de tablas

In [ ]:
tablas_data= {
    'Geometrica_A' : Geometrica_A,
    'Geometrica_B' : Geometrica_B,
    'Geometrica_C' : Geometrica_C,
    'Utilizacion' : Utilizacion,
    'Calzadas' : Calzadas,
    'Carriles' : Carriles,
    'Material' : Material,
    'Estado_Via' : Estado_Via,
    'Condiciones' : Condiciones,
    'Iluminacion_A' : Iluminacion_A,
    'Agente_Transito' : Agente_Transito,
    'Semaforo' : Semaforo,
    'Clase_Vh' : Clase_Vh,
    'Servicio' : Servicio,
    'Modalidad' : Modalidad,
    'Describe_Causa' : Describe_Causa,
    'Tipo_Causa' : Tipo_Causa,
    'Tiempo': Tiempo,
    'Horario' : Horario,
    'Gravedad' : Gravedad,
    'Clase_Accidente' : Clase_Accidente,
    'Localidad' : Localidad,
    'Condicion_Actor' : Condicion_Actor,
    'Estado_ActorVial' : Estado_ActorVial,
    'Genero' : Genero,
    'Accidente': Accidente_df,
    'Actor_Vial': Actor_Vial_df,
    'Via': Via_df,
    'Vehiculo': Vehiculo_df,
    'Causa': Causa_df
}

### 4.2. Creación de Script de establecimiento de llaves primarias

In [ ]:
# Define las llaves primarias
llaves_primarias = {
    'Geometrica_A': 'COD_GEOMETRICA_A',
    'Geometrica_B': 'COD_GEOMETRICA_B',
    'Geometrica_C': 'COD_GEOMETRICA_C',
    'Utilizacion': 'COD_UTILIZACION',
    'Calzadas': 'COD_CALZADAS',
    'Carriles': 'COD_CARRILES',
    'Material': 'COD_MATERIAL',
    'Estado_Via': 'COD_ESTADO',
    'Condiciones': 'COD_CONDICION',
    'Iluminacion_A': 'COD_ILUMINACION_A',
    'Agente_Transito': 'COD_AGENTE_TRANSITO',
    'Semaforo': 'COD_SEMAFORO',
    'Clase_Vh': 'COD_CLASE',
    'Servicio': 'COD_SERVICIO',
    'Modalidad': 'COD_MODALIDAD',
    'Condicion_Actor': 'COD_CONDICION_ACTOR',
    'Genero': 'COD_GENERO',
    'Estado_ActorVial': 'COD_ESTADO',
    'Describe_Causa': 'CODIGO_CAUSA',
    'Tipo_Causa': 'TIPO',
    'Tiempo': 'FECHA_OCURRENCIA_ACC',
    'Horario': 'HORA_OCURRENCIA_ACC',
    'Gravedad': 'COD_GRAVEDAD',
    'Clase_Accidente': 'COD_CLASE_ACC',
    'Localidad': 'COD_LOCALIDAD',
    'Accidente': 'CODIGO_ACCIDENTE',
}

### 4.3. Creación de función para generar los archivos a ejecutar en PostgreSQL para crear las tablas

In [ ]:
# Definir creación de tablas
def generate_sql_create_table_script(table_name, df):
    columns = df.columns.tolist()
    script_sql_creacion = f"CREATE TABLE {table_name} (\n"

    for i, col in enumerate(columns):
        # Validar el tipo de campo y longitud
        column_type = "VARCHAR"  # Se puede ajustar los tipos según la necesidad
        max_length = df[col].apply(str).str.len().max()

        # Ajustar el tipo de campo según la naturaleza de los datos
        if ((df[col].dtype == 'int64' or df[col].dtype == 'float64') and df[col].name != 'LATITUD') or ((df[col].dtype == 'int64' or df[col].dtype == 'float64') and df[col].name != 'LONGITUD'):
            column_type = "INTEGER"
        elif df[col].dtype == 'float64':
            column_type = "FLOAT"

        # Limitar la longitud de VARCHAR a un valor razonable
        max_length = min(max_length, 255) if column_type == "VARCHAR" else None

        script_sql_creacion += f"  {col} {column_type}"
        if max_length:
            script_sql_creacion += f"({max_length})"
        if table_name in llaves_primarias:
          script_sql_creacion += "," if i < len(columns) else ""
        else:
          script_sql_creacion += "," if i < len(columns) - 1 else ""
        script_sql_creacion += "\n"

    if table_name in llaves_primarias:
      script_sql_creacion += f"  PRIMARY KEY ({llaves_primarias[table_name]})\n"

    script_sql_creacion += ");\n\n"

    return script_sql_creacion

### 4.4. Creación de los archivos SQL para crear las tablas

In [ ]:
# Iterar a través de las tablas en tablas_data y generar los scripts SQL de creación
script_sql_creacion = ""
for table_name, df in tablas_data.items():
    script_sql_creacion += generate_sql_create_table_script(table_name, df)

In [ ]:
# Guardar el script SQL en un archivo
output_file_path = 'Script_creacion_tablas.sql'
with open(output_file_path, 'w') as file:
    file.write(script_sql_creacion)

### 4.5. Creación Script Relaciones

In [ ]:
# Diccionario que almacena las relaciones foráneas
relaciones_foraneas = {
    'Via': {'CODIGO_ACCIDENTE': 'Accidente', 'COD_GEOMETRICA_A':'Geometrica_A',
            'COD_GEOMETRICA_B':'Geometrica_B','COD_GEOMETRICA_C':'Geometrica_C',
            'COD_UTILIZACION':'Utilizacion', 'COD_CALZADAS': 'Calzadas', 'COD_CARRILES':'Carriles',
            'COD_MATERIAL':'Material', 'COD_ESTADO': 'Estado_Via', 'COD_CONDICION':'Condiciones',
            'COD_ILUMINACION_A':'Iluminacion_A', 'COD_AGENTE_TRANSITO':'Agente_Transito',
            'COD_SEMAFORO':'Semaforo'},
    'Vehiculo': {'CODIGO_ACCIDENTE': 'Accidente','COD_CLASE': 'Clase_Vh',
                 'COD_SERVICIO': 'Servicio', 'COD_MODALIDAD':'Modalidad'},
    'Actor_Vial': {'CODIGO_ACCIDENTE': 'Accidente','COD_GENERO':'Genero', 'COD_ESTADO': 'Estado_ActorVial',
                   'COD_CONDICION_ACTOR':'Condicion_Actor'},
    'Causa': {'CODIGO_ACCIDENTE':'Accidente','CODIGO_CAUSA':'Describe_Causa', 'TIPO':'Tipo_Causa'},
    'Accidente': {'FECHA_OCURRENCIA_ACC':'Tiempo', 'HORA_OCURRENCIA_ACC':'Horario',
                  'COD_GRAVEDAD':'Gravedad', 'COD_CLASE_ACC':'Clase_Accidente',
                  'COD_LOCALIDAD':'Localidad'},
}

In [ ]:
# Función para generar un script SQL de relaciones foráneas
def generar_script_relaciones_foraneas(relaciones_foraneas):
    script_relaciones = ""
    for tabla, relaciones in relaciones_foraneas.items():
        for columna_foranea, tabla_referenciada in relaciones.items():
            script_relaciones += f"ALTER TABLE {tabla} ADD FOREIGN KEY ({columna_foranea}) REFERENCES {tabla_referenciada}({columna_foranea});\n"
    return script_relaciones

In [ ]:
# Genera el script de relaciones foráneas
script_relaciones_foraneas = generar_script_relaciones_foraneas(relaciones_foraneas)

In [ ]:
# Guarda el script en un archivo
archivo_script_relaciones = 'Script_relaciones_foraneas.sql'
with open(archivo_script_relaciones, 'w') as archivo:
    archivo.write(script_relaciones_foraneas)

### 4.6 Ajuste previo columnas de texto

#### 4.6.1 Ajuste tabla Geometrica A

In [ ]:
#Se incluyen las comas en las columnas de texto
Geometrica_A['GEOMETRICA_A'] = Geometrica_A['GEOMETRICA_A'].apply(lambda x: f"'{x}'")

In [ ]:
print(Geometrica_A.head())

  GEOMETRICA_A  COD_GEOMETRICA_A
0      'RECTA'                 1
1      'CURVA'                 2


#### 4.6.2 Ajuste tabla Geometrica B

In [ ]:
#Se incluyen las comas en las columnas de texto
Geometrica_B['GEOMETRICA_B'] = Geometrica_B['GEOMETRICA_B'].apply(lambda x: f"'{x}'")

In [ ]:
print(Geometrica_B.head())

  GEOMETRICA_B  COD_GEOMETRICA_B
0      'PLANO'                 1
1  'PENDIENTE'                 2


#### 4.6.3 Ajuste tabla Geometrica C

In [ ]:
#Se incluyen las comas en las columnas de texto
Geometrica_C['GEOMETRICA_C'] = Geometrica_C['GEOMETRICA_C'].apply(lambda x: f"'{x}'")

In [ ]:
print(Geometrica_C.head())

      GEOMETRICA_C  COD_GEOMETRICA_C
0      'CON ANDEN'                 1
1     'CON BERMAS'                 2
2  'BAHIA DE EST.'                 3


#### 4.6.4 Ajuste tabla Utilizacion

In [ ]:
#Se incluyen las comas en las columnas de texto
Utilizacion['UTILIZACION'] = Utilizacion['UTILIZACION'].apply(lambda x: f"'{x}'")

In [ ]:
print(Utilizacion.head())

       UTILIZACION  COD_UTILIZACION
0  'DOBLE SENTIDO'                1
1     'UN SENTIDO'                2
2     'REVERSIBLE'                3
3    'CONTRAFLUJO'                4
4       'CICLOVIA'                5


#### 4.6.5 Ajuste tabla Calzadas

In [ ]:
#Se incluyen las comas en las columnas de texto
Calzadas['CALZADAS'] = Calzadas['CALZADAS'].apply(lambda x: f"'{x}'")

In [ ]:
print(Calzadas.head())

         CALZADAS  COD_CALZADAS
0           'UNA'             1
1    'TRES O MAS'             2
2           'DOS'             3
3      'VARIABLE'             4
4  'CUATRO O MAS'             5


#### 4.6.6 Ajuste tabla Carriles

In [ ]:
#Se incluyen las comas en las columnas de texto
Carriles['CARRILES'] = Carriles['CARRILES'].apply(lambda x: f"'{x}'")

In [ ]:
print(Carriles.head())

         CARRILES  COD_CARRILES
0           'UNO'             1
1           'DOS'             2
2    'TRES O MAS'             3
3      'VARIABLE'             4
4  'CUATRO O MAS'             5


#### 4.6.7 Ajuste tabla Material

In [ ]:
#Se incluyen las comas en las columnas de texto
Material['MATERIAL'] = Material['MATERIAL'].apply(lambda x: f"'{x}'")

In [ ]:
print(Material.head())

     MATERIAL  COD_MATERIAL
0  'CONCRETO'             1
1   'ASFALTO'             2
2  'AFIRMADO'             3
3   'ADOQUIN'             4
4    'TIERRA'             5


#### 4.6.8 Ajuste tabla Estado_Via

In [ ]:
#Se incluyen las comas en las columnas de texto
Estado_Via['ESTADO'] = Estado_Via['ESTADO'].apply(lambda x: f"'{x}'")

In [ ]:
print(Estado_Via.head(9))

            ESTADO  COD_ESTADO
0          'BUENO'           1
1      'DERRUMBES'           2
2     'CON HUECOS'           3
3         'RIZADA'           4
4   'HUNDIMIENTOS'           5
5  'EN REPARACION'           6
6       'PARCHADA'           7
7       'FISURADA'           8
8       'INUNDADA'           9


#### 4.6.9 Ajuste tabla Condiciones

In [ ]:
#Se incluyen las comas en las columnas de texto
Condiciones['CONDICIONES'] = Condiciones['CONDICIONES'].apply(lambda x: f"'{x}'")

In [ ]:
print(Condiciones.head(8))

                CONDICIONES  COD_CONDICION
0                    'SECA'              1
1         'MATERIAL SUELTO'              2
2                  'HUMEDA'              3
3  'ALCANTARILLA DESTAPADA'              4
4                    'LODO'              5
5                  'ACEITE'              6
6                    'OTRA'              7
7       'MATERIAL ORGANICO'              8


#### 4.6.10 Ajuste tabla Iluminacion A

In [ ]:
#Se incluyen las comas en las columnas de texto
Iluminacion_A['ILUMINACION_A'] = Iluminacion_A['ILUMINACION_A'].apply(lambda x: f"'{x}'")

In [ ]:
print(Iluminacion_A.head())

  ILUMINACION_A  COD_ILUMINACION_A
0         'SIN'                  1
1         'CON'                  2


#### 4.6.11 Ajuste tabla Agente Transito

In [ ]:
#Se incluyen las comas en las columnas de texto
Agente_Transito['AGENTE_TRANSITO'] = Agente_Transito['AGENTE_TRANSITO'].apply(lambda x: f"'{x}'")

In [ ]:
print(Agente_Transito.head())

  AGENTE_TRANSITO  COD_AGENTE_TRANSITO
0            'NO'                    1
1            'SI'                    2


#### 4.6.12 Ajuste tabla Semaforo

In [ ]:
#Se incluyen las comas en las columnas de texto
Semaforo['SEMAFORO'] = Semaforo['SEMAFORO'].apply(lambda x: f"'{x}'")

In [ ]:
print(Semaforo.head(6))

         SEMAFORO  COD_SEMAFORO
0      'OPERANDO'             1
1        'OCULTO'             2
2  'INTERMITENTE'             3
3     'CON DAÑOS'             4
4       'APAGADO'             5
5       'NINGUNO'             6


#### 4.6.13  Ajuste tabla Via

In [ ]:
#Se incluyen las comas en las columnas de texto
Via_df['CODIGO'] = Via_df['CODIGO'].apply(lambda x: f"'{x}'")

In [ ]:
#Se transforman a texto las columnas con NaN para reemplazarlas por null
Via_df['COD_GEOMETRICA_A'] = Via_df['COD_GEOMETRICA_A'].map(str)
Via_df['COD_GEOMETRICA_B'] = Via_df['COD_GEOMETRICA_B'].map(str)
Via_df['COD_GEOMETRICA_C'] = Via_df['COD_GEOMETRICA_C'].map(str)
Via_df['COD_UTILIZACION'] = Via_df['COD_UTILIZACION'].map(str)
Via_df['COD_CALZADAS'] = Via_df['COD_CALZADAS'].map(str)
Via_df['COD_CARRILES'] = Via_df['COD_CARRILES'].map(str)
Via_df['COD_MATERIAL'] = Via_df['COD_MATERIAL'].map(str)
Via_df['COD_ESTADO'] = Via_df['COD_ESTADO'].map(str)
Via_df['COD_CONDICION'] = Via_df['COD_CONDICION'].map(str)
Via_df['COD_ILUMINACION_A'] = Via_df['COD_ILUMINACION_A'].map(str)
Via_df['COD_AGENTE_TRANSITO'] = Via_df['COD_AGENTE_TRANSITO'].map(str)
Via_df['COD_SEMAFORO'] = Via_df['COD_SEMAFORO'].map(str)

In [ ]:
#Se selecciona solo la primer posición de las columnas transformads en la celda anterior
Via_df['COD_GEOMETRICA_A'] = Via_df['COD_GEOMETRICA_A'].str[:1]
Via_df['COD_GEOMETRICA_B'] = Via_df['COD_GEOMETRICA_B'].str[:1]
Via_df['COD_GEOMETRICA_C'] = Via_df['COD_GEOMETRICA_C'].str[:1]
Via_df['COD_UTILIZACION'] = Via_df['COD_UTILIZACION'].str[:1]
Via_df['COD_CALZADAS'] = Via_df['COD_CALZADAS'].str[:1]
Via_df['COD_CARRILES'] = Via_df['COD_CARRILES'].str[:1]
Via_df['COD_MATERIAL'] = Via_df['COD_MATERIAL'].str[:1]
Via_df['COD_ESTADO'] = Via_df['COD_ESTADO'].str[:1]
Via_df['COD_CONDICION'] = Via_df['COD_CONDICION'].str[:1]
Via_df['COD_ILUMINACION_A'] = Via_df['COD_ILUMINACION_A'].str[:1]
Via_df['COD_AGENTE_TRANSITO'] = Via_df['COD_AGENTE_TRANSITO'].str[:1]
Via_df['COD_SEMAFORO'] = Via_df['COD_SEMAFORO'].str[:1]

In [ ]:
#Se reemplazan las n por null
Via_df = Via_df.replace('n', 'null')

In [ ]:
print(Via_df.head(6))

   CODIGO_ACCIDENTE COD_GEOMETRICA_A COD_GEOMETRICA_B COD_GEOMETRICA_C  \
0            511950                1                1                1   
1            511947                1                1                1   
2            511947                1                1                2   
3            511951                1                1                1   
4            511953                1                1                1   
5            511955                1                1                2   

  COD_UTILIZACION COD_CALZADAS COD_CARRILES COD_MATERIAL COD_ESTADO  \
0               1            1            1            1          1   
1               1            2            2            2          1   
2               2            1            1            2          1   
3               1            1            2            2          1   
4               1            3            2            2          1   
5               2            1            1            

#### 4.6.14 Ajuste tabla Tiempo

In [ ]:
#Se incluyen las comas en las columnas de texto
Tiempo['FECHA_OCURRENCIA_ACC'] = Tiempo['FECHA_OCURRENCIA_ACC'].apply(lambda x: f"'{x}'")
Tiempo['TXT_MES_OCURRENCIA'] = Tiempo['TXT_MES_OCURRENCIA'].apply(lambda x: f"'{x}'")
Tiempo['TXT_DIA_OCURRENCIA'] = Tiempo['TXT_DIA_OCURRENCIA'].apply(lambda x: f"'{x}'")

In [ ]:
print(Tiempo.head())

  FECHA_OCURRENCIA_ACC  ANO_OCURRENCIA  NRO_MES_OCURRENCIA TXT_MES_OCURRENCIA  \
0         '2017-06-12'            2017                   6             'June'   
1         '2022-10-29'            2022                  10          'October'   
2         '2012-12-15'            2012                  12         'December'   
3         '2022-11-01'            2022                  11         'November'   
4         '2014-10-26'            2014                  10          'October'   

   NRO_DIA_OCURRENCIA TXT_DIA_OCURRENCIA  TRIMESTRE_OCURRENCIA  
0                  12           'Monday'                     2  
1                  29         'Saturday'                     4  
2                  15         'Saturday'                     4  
3                   1          'Tuesday'                     4  
4                  26           'Sunday'                     4  


#### 4.6.15 Ajuste tabla Horario

In [ ]:
#Se incluyen las comas en las columnas de texto
Horario['HORA_OCURRENCIA_ACC'] = Horario['HORA_OCURRENCIA_ACC'].apply(lambda x: f"'{x}'")
Horario['HORA_OCURRENCIA'] = Horario['HORA_OCURRENCIA'].apply(lambda x: f"'{x}'")
Horario['JORNADA_OCURRENCIA'] = Horario['JORNADA_OCURRENCIA'].apply(lambda x: f"'{x}'")

In [ ]:
print(Horario.head())

  HORA_OCURRENCIA_ACC HORA_OCURRENCIA JORNADA_OCURRENCIA
0          '05:30:00'            '05'        'Madrugada'
1          '10:55:00'            '10'           'Mañana'
2          '20:30:00'            '20'            'Noche'
3          '14:58:00'            '14'            'Tarde'
4          '21:25:00'            '21'            'Noche'


#### 4.6.16 Ajuste tabla Gravedad

In [ ]:
#Se incluyen las comas en las columnas de texto
Gravedad['GRAVEDAD'] = Gravedad['GRAVEDAD'].apply(lambda x: f"'{x}'")

In [ ]:
print(Gravedad.head())

        GRAVEDAD  COD_GRAVEDAD
0   'SOLO DANOS'           1.0
1  'CON HERIDOS'           2.0
2  'CON MUERTOS'           3.0


#### 4.6.17 Ajuste tabla Clase_Accidente

In [ ]:
#Se incluyen las comas en las columnas de texto
Clase_Accidente['CLASE_ACC'] = Clase_Accidente['CLASE_ACC'].apply(lambda x: f"'{x}'")

In [ ]:
print(Clase_Accidente.head())

       CLASE_ACC  COD_CLASE_ACC
0       'CHOQUE'            1.0
1    'ATROPELLO'            2.0
2   'AUTOLESION'            3.0
3  'VOLCAMIENTO'            4.0
4         'OTRO'            5.0


#### 4.6.18 Ajuste tabla Localidad

In [ ]:
#Se incluyen las comas en las columnas de texto
Localidad['LOCALIDAD'] = Localidad['LOCALIDAD'].apply(lambda x: f"'{x}'")

In [ ]:
print(Localidad.head(20))

               LOCALIDAD  COD_LOCALIDAD
0             'ENGATIVA'            1.0
1            'CHAPINERO'            2.0
2                 'SUBA'            3.0
3             'FONTIBON'            4.0
4       'CIUDAD BOLIVAR'            5.0
5         'LOS MARTIRES'            6.0
6             'SANTA FE'            7.0
7                 'BOSA'            8.0
8              'USAQUEN'            9.0
9              'KENNEDY'           10.0
10  'RAFAEL URIBE URIBE'           11.0
11         'TEUSAQUILLO'           12.0
12       'SAN CRISTOBAL'           13.0
13       'PUENTE ARANDA'           14.0
14          'TUNJUELITO'           15.0
15      'ANTONIO NARINO'           16.0
16                'USME'           17.0
17      'BARRIOS UNIDOS'           18.0
18          'CANDELARIA'           19.0
19             'SUMAPAZ'           20.0


#### 4.6.19 Ajuste tabla Condición_Actor

In [ ]:
#Se incluyen las comas en las columnas de texto
Condicion_Actor['CONDICION'] = Condicion_Actor['CONDICION'].apply(lambda x: f"'{x}'")

In [ ]:
print(Condicion_Actor.head(6))

           CONDICION  COD_CONDICION_ACTOR
0        'CONDUCTOR'                  1.0
1     'MOTOCICLISTA'                  2.0
2         'CICLISTA'                  3.0
3         'PASAJERO'                  4.0
4           'PEATON'                  5.0
5  'SIN INFORMACION'                  6.0


#### 4.6.20 Ajuste tabla Estado

In [ ]:
#Se incluyen las comas en las columnas de texto
Estado_ActorVial['ESTADO'] = Estado_ActorVial['ESTADO'].apply(lambda x: f"'{x}'")

In [ ]:
print(Estado_ActorVial.head())

     ESTADO  COD_ESTADO
0   'ILESO'         1.0
1  'HERIDO'         2.0
2  'MUERTO'         3.0


#### 4.6.21 Ajuste tabla Genero

In [ ]:
#Se incluyen las comas en las columnas de texto
Genero['GENERO'] = Genero['GENERO'].apply(lambda x: f"'{x}'")

In [ ]:
print(Genero.head())

              GENERO  COD_GENERO
0        'MASCULINO'         1.0
1         'FEMENINO'         2.0
2  'SIN INFORMACION'         3.0


#### 4.6.22 Ajuste tabla Actor Vial

In [ ]:
#Se incluyen las comas en las columnas de texto
Actor_Vial_df['FECHA_NACIMIENTO'] = Actor_Vial_df['FECHA_NACIMIENTO'].apply(lambda x: f"'{x}'")
Actor_Vial_df['CODIGO'] = Actor_Vial_df['CODIGO'].apply(lambda x: f"'{x}'")

In [ ]:
#Se transforman a texto las columnas con NaN para reemplazarlas por null
Actor_Vial_df['CODIGO_ACCIDENTADO'] = Actor_Vial_df['CODIGO_ACCIDENTADO'].map(str)
Actor_Vial_df['COD_CONDICION_ACTOR'] = Actor_Vial_df['COD_CONDICION_ACTOR'].map(str)
Actor_Vial_df['COD_ESTADO'] = Actor_Vial_df['COD_ESTADO'].map(str)
Actor_Vial_df['COD_GENERO'] = Actor_Vial_df['COD_GENERO'].map(str)

In [ ]:
#Se selecciona solo la primer posición de las columnas transformads en la celda anterior
Actor_Vial_df['CODIGO_ACCIDENTADO'] = Actor_Vial_df['CODIGO_ACCIDENTADO'].str[:1]
Actor_Vial_df['COD_CONDICION_ACTOR'] = Actor_Vial_df['COD_CONDICION_ACTOR'].str[:1]
Actor_Vial_df['COD_ESTADO'] = Actor_Vial_df['COD_ESTADO'].str[:1]
Actor_Vial_df['COD_GENERO'] = Actor_Vial_df['COD_GENERO'].str[:1]

In [ ]:
#Se reemplazan las n por null
Actor_Vial_df = Actor_Vial_df.replace('n', 'null')
Actor_Vial_df = Actor_Vial_df.replace('\'NaT\'', 'null')

In [ ]:
print(Actor_Vial_df.head())

  CODIGO_ACCIDENTADO COD_CONDICION_ACTOR COD_ESTADO COD_GENERO  \
0                  1                   1          1          1   
1                  1                   1          1          1   
2                  1                   1          1          1   
3                  1                   1          1          2   
4                  1                   1          1          1   

  FECHA_NACIMIENTO        CODIGO  CODIGO_ACCIDENTE  
0     '1991-04-28'  '10528705-1'          10528705  
1     '1956-04-01'  '10528705-2'          10528705  
2     '1997-06-23'  '10528709-1'          10528709  
3     '1985-05-25'  '10528709-2'          10528709  
4     '1979-10-04'  '10528708-1'          10528708  


#### 4.6.23 Ajuste tabla Descripcion_Causa

In [ ]:
#Se incluyen las comas en las columnas de texto
Describe_Causa['NOMBRE'] = Describe_Causa['NOMBRE'].apply(lambda x: f"'{x}'")

In [ ]:
print(Describe_Causa.head(6))

   CODIGO_CAUSA                                          NOMBRE
0         157.0                                          'OTRA'
1         121.0            'NO MANTENER DISTANCIA DE SEGURIDAD'
2         143.0  'PONER EN MARCHA UN VEHICULO SIN PRECAUCIONES'
3         139.0          'NO UTILIZAR - IMPERICIA EN EL MANEJO'
4         103.0                            'ADELANTAR CERRANDO'
5         104.0                      'ADELANTAR INVADIENDO VIA'


#### 4.6.24 Ajuste tabla Tipo_Causa

In [ ]:
#Se incluyen las comas en las columnas de texto
Tipo_Causa['TIPO'] = Tipo_Causa['TIPO'].apply(lambda x: f"'{x}'")
Tipo_Causa['TIPO_CAUSA'] = Tipo_Causa['TIPO_CAUSA'].apply(lambda x: f"'{x}'")

In [ ]:
print(Tipo_Causa.head(6))

   TIPO   TIPO_CAUSA
0  'CG'  'CONDUCTOR'
1  'VI'        'VIA'
2  'CE'  'CONDUCTOR'
3  'VH'   'VEHICULO'
4  'PE'     'PEATON'
5  'PA'   'PASAJERO'


#### 4.6.25 Ajuste tabla Causa

In [ ]:
#Se reemplazan las n por null
Causa_df = Causa_df.replace('nan', 'null')

In [ ]:
#Se incluyen las comas en las columnas de texto
Causa_df['TIPO'] = Causa_df['TIPO'].apply(lambda x: f"'{x}'")
Causa_df['CODIGO_AC_VH'] = Causa_df['CODIGO_AC_VH'].apply(lambda x: f"'{x}'")

In [ ]:
#Se transforman a texto las columnas con NaN para reemplazarlas por null
Causa_df['CODIGO_CAUSA'] = Causa_df['CODIGO_CAUSA'].map(str)

In [ ]:
#Conservar lo que está a la izquierda del punto en la columna Codigo_Causa
Causa_df['CODIGO_CAUSA'] = Causa_df['CODIGO_CAUSA'].apply(lambda x: x.split('.')[0])

In [ ]:
#Se reemplazan los nan por null
Causa_df = Causa_df.replace('nan', 'null')
Causa_df = Causa_df.replace('\'nan\'', 'null')

In [ ]:
Causa_df.head()

,CODIGO_ACCIDENTE,CODIGO_CAUSA,TIPO,CODIGO_AC_VH
0,10558992,157,'CG','10558992-1'
1,10558992,157,'CG','10558992-2'
2,10558993,121,'CG','10558993-2'
3,10558993,157,'CG','10558993-2'
4,10558994,143,'CG','10558994-1'


#### 4.6.26 Ajuste tabla Clase_Vehiculo

In [ ]:
#Se incluyen las comas en las columnas de texto
Clase_Vh['CLASE'] = Clase_Vh['CLASE'].apply(lambda x: f"'{x}'")

In [ ]:
print(Clase_Vh.head(27))

                CLASE  COD_CLASE
0       'MOTOCICLETA'        1.0
1         'AUTOMOVIL'        2.0
2               'BUS'        3.0
3      'TRACTOCAMION'        4.0
4    'CAMION, FURGON'        5.0
5         'CAMIONETA'        6.0
6          'MICROBUS'        7.0
7           'CAMPERO'        8.0
8            'BUSETA'        9.0
9          'VOLQUETA'       10.0
10        'MOTOCARRO'       11.0
11        'BICICLETA'       12.0
12  'NO IDENTIFICADO'       13.0
13  'TRACCION ANIMAL'       14.0
14             'OTRO'       15.0
15        'MOTOCICLO'       16.0
16    'M. INDUSTRIAL'       17.0
17      'M. AGRICOLA'       18.0
18   'BUS ARTICULADO'       19.0
19  'BUS ALIMENTADOR'       20.0
20            'METRO'       21.0
21       'AMBULANCIA'       22.0
22             'TREN'       23.0
23         'BICITAXI'       24.0
24       'CUATRIMOTO'       25.0
25         'REMOLQUE'       26.0
26    'SEMI-REMOLQUE'       27.0


#### 4.6.27 Ajuste tabla Servicio

In [ ]:
#Se incluyen las comas en las columnas de texto
Servicio['SERVICIO'] = Servicio['SERVICIO'].apply(lambda x: f"'{x}'")

In [ ]:
print(Servicio.head(5))

            SERVICIO  COD_SERVICIO
0       'PARTICULAR'           1.0
1          'PUBLICO'           2.0
2          'OFICIAL'           3.0
3      'DIPLOMATICO'           4.0
4  'SIN INFORMACION'           5.0


#### 4.6.28 Ajuste tabla Modalidad

In [ ]:
#Se incluyen las comas en las columnas de texto
Modalidad['MODALIDAD'] = Modalidad['MODALIDAD'].apply(lambda x: f"'{x}'")

In [ ]:
print(Modalidad.head(12))

                           MODALIDAD  COD_MODALIDAD
0               'PASAJEROS - MASIVO'            1.0
1      'CARGA - MERCANCIA PELIGROSA'            2.0
2                     'CARGA-PESADA'            3.0
3           'PASAJEROS - INDIVIDUAL'            4.0
4            'PASAJEROS - COLECTIVO'            5.0
5     'PASAJEROS - ESPECIAL TURISMO'            6.0
6   'PASAJEROS - ESPECIAL OCASIONAL'            7.0
7                            'MIXTO'            8.0
8     'PASAJEROS - ESPECIAL ESCOLAR'            9.0
9              'CARGA - EXTRAPESADA'           10.0
10  'PASAJEROS - ESPECIAL ASALARIAD'           11.0
11       'CARGA - EXTRADIMENSIONADA'           12.0


#### 4.6.29 Ajuste tabla Vehiculo

In [ ]:
# Se eliminan las comillas sencillas existentes en la columna placa
Vehiculo_df['PLACA'] = Vehiculo_df['PLACA'].str.replace("'", "")
Vehiculo_df['PLACA'] = Vehiculo_df['PLACA'].str.replace("HERNANDEZ", "null")

In [ ]:
#Se incluyen las comas en las columnas de texto
Vehiculo_df['PLACA'] = Vehiculo_df['PLACA'].apply(lambda x: f"'{x}'")
Vehiculo_df['ENFUGA'] = Vehiculo_df['ENFUGA'].apply(lambda x: f"'{x}'")
Vehiculo_df['CODIGO'] = Vehiculo_df['CODIGO'].apply(lambda x: f"'{x}'")

In [ ]:
#Se transforman a texto las columnas con NaN para reemplazarlas por null
Vehiculo_df['COD_CLASE'] = Vehiculo_df['COD_CLASE'].map(str)
Vehiculo_df['COD_SERVICIO'] = Vehiculo_df['COD_SERVICIO'].map(str)
Vehiculo_df['COD_MODALIDAD'] = Vehiculo_df['COD_MODALIDAD'].map(str)

In [ ]:
#Se selecciona solo la primer posición de las columnas transformads en la celda anterior
Vehiculo_df['COD_CLASE'] = Vehiculo_df['COD_CLASE'].str[:1]
Vehiculo_df['COD_SERVICIO'] = Vehiculo_df['COD_SERVICIO'].str[:1]
Vehiculo_df['COD_MODALIDAD'] = Vehiculo_df['COD_MODALIDAD'].str[:1]

In [ ]:
#Se reemplazan las n por null
Vehiculo_df = Vehiculo_df.replace('n', 'null')
Vehiculo_df = Vehiculo_df.replace('\'nan\'', 'null')

In [ ]:
print(Vehiculo_df.head(4))

      PLACA COD_CLASE COD_SERVICIO COD_MODALIDAD ENFUGA      CODIGO  \
0  'BWH61C'         1            1          null    'N'  '511951-1'   
1  'BSU084'         2            1          null    'N'  '511950-1'   
2  'CUW805'         2            1          null    'N'  '511947-1'   
3  'YHV59C'         1            1          null    'N'  '511947-2'   

   CODIGO_ACCIDENTE  
0            511951  
1            511950  
2            511947  
3            511947  


#### 4.6.30 Ajuste tabla Accidente

In [ ]:
#Se incluyen las comas en las columnas de texto
Accidente_df['FECHA_OCURRENCIA_ACC'] = Accidente_df['FECHA_OCURRENCIA_ACC'].apply(lambda x: f"'{x}'")
Accidente_df['HORA_OCURRENCIA_ACC'] = Accidente_df['HORA_OCURRENCIA_ACC'].apply(lambda x: f"'{x}'")
Accidente_df['DIRECCION'] = Accidente_df['DIRECCION'].apply(lambda x: f"'{x}'")

In [ ]:
#Se transforman a texto las columnas con NaN para reemplazarlas por null
Accidente_df['COD_GRAVEDAD'] = Accidente_df['COD_GRAVEDAD'].map(str)
Accidente_df['COD_CLASE_ACC'] = Accidente_df['COD_CLASE_ACC'].map(str)
Accidente_df['COD_LOCALIDAD'] = Accidente_df['COD_LOCALIDAD'].map(str)
Accidente_df['COD_GRAVEDAD'] = Accidente_df['COD_GRAVEDAD'].map(str)

In [ ]:
#Se selecciona solo la primer posición de las columnas transformads en la celda anterior
Accidente_df['COD_GRAVEDAD'] = Accidente_df['COD_GRAVEDAD'].str[:1]
Accidente_df['COD_CLASE_ACC'] = Accidente_df['COD_CLASE_ACC'].str[:1]
Accidente_df['COD_LOCALIDAD'] = Accidente_df['COD_LOCALIDAD'].str[:1]
Accidente_df['COD_GRAVEDAD'] = Accidente_df['COD_GRAVEDAD'].str[:1]

In [ ]:
#Se reemplazan las n por null
Accidente_df = Accidente_df.replace('n', 'null')
Accidente_df = Accidente_df.replace('\'NaT\'', 'null')
Accidente_df = Accidente_df.replace('', 'null')

In [ ]:
# Visualizar el nuevo DataFrame con las columnas seleccionadas
print(Accidente_df.head())

   CODIGO_ACCIDENTE FECHA_OCURRENCIA_ACC HORA_OCURRENCIA_ACC  \
0           4484660         '2017-06-12'          '05:30:00'   
1          10583520         '2022-10-29'          '10:55:00'   
2            449558         '2012-12-15'          '20:30:00'   
3          10583513         '2022-11-01'          '14:58:00'   
4            513150         '2014-10-26'          '21:25:00'   

                      DIRECCION COD_GRAVEDAD COD_CLASE_ACC COD_LOCALIDAD  \
0  'AV AVENIDA BOYACA-CL 79 02'            1             1             1   
1      'KR 111 A  - CL 79   02'            1             1             1   
2               'KR 4-CL 91 02'            1             1             2   
3        'KR 72  - CL 148   02'            2             1             3   
4             'AC 72-KR 68F 02'            2             2             1   

    LATITUD   LONGITUD  
0  4.693807 -74.090924  
1  4.717638 -74.119697  
2  4.669288 -74.040677  
3  4.737015 -74.066101  
4  4.679568 -74.087407  


#### 4.6.31 Volver a crear tablas

In [ ]:
tablas_data= {
    'Geometrica_A' : Geometrica_A,
    'Geometrica_B' : Geometrica_B,
    'Geometrica_C' : Geometrica_C,
    'Utilizacion' : Utilizacion,
    'Calzadas' : Calzadas,
    'Carriles' : Carriles,
    'Material' : Material,
    'Estado_Via' : Estado_Via,
    'Condiciones' : Condiciones,
    'Iluminacion_A' : Iluminacion_A,
    'Agente_Transito' : Agente_Transito,
    'Semaforo' : Semaforo,
    'Clase_Vh' : Clase_Vh,
    'Servicio' : Servicio,
    'Modalidad' : Modalidad,
    'Describe_Causa' : Describe_Causa,
    'Tipo_Causa' : Tipo_Causa,
    'Tiempo': Tiempo,
    'Horario' : Horario,
    'Gravedad' : Gravedad,
    'Clase_Accidente' : Clase_Accidente,
    'Localidad' : Localidad,
    'Condicion_Actor' : Condicion_Actor,
    'Estado_ActorVial' : Estado_ActorVial,
    'Genero' : Genero,
    'Accidente': Accidente_df,
    'Actor_Vial': Actor_Vial_df,
    'Via': Via_df,
    'Vehiculo': Vehiculo_df,
    'Causa': Causa_df
}

### 4.7. Creación Scripts cargue de BBDD

In [ ]:
# Define el directorio donde se guardarán los archivos SQL de inserción
sql_directory = 'Scripts_cargue_BBDD'

# Asegúrate de que la carpeta exista o créala
os.makedirs(sql_directory, exist_ok=True)

In [ ]:
def generate_sql_insert_script(tabla_nombre, df):
    # Obtener los nombres de las columnas del DataFrame en la lista columns
    columns = df.columns.tolist()

    # Crear el script de inserción SQL
    sql_insert_script = f"INSERT INTO {tabla_nombre} ({', '.join(columns)}) VALUES\n"
    sql_insert_script += ',\n'.join([f"({', '.join(map(str, row))})" for row in df.values])
    sql_insert_script += ';\n'

    return sql_insert_script


In [ ]:
# Se genera un script de inserción para todas las tablas
sql_script = ''
for tabla_nombre, df in tablas_data.items():
    sql_script += generate_sql_insert_script(tabla_nombre, df)

# Guardar el script SQL en un archivo único
output_file = os.path.join(sql_directory, 'insert_all_tables.sql')
with open(output_file, 'w') as file:
    file.write(sql_script)

### 4.8. Descarga de archivos .SQL

In [ ]:
from zipfile import ZipFile
from google.colab import files

In [ ]:
archivos_sql = ['Script_creacion_tablas.sql', 'Script_relaciones_foraneas.sql']

# Agregar archivos individuales de Script_cargue_BBDD al archivo ZIP
archivos_cargue_BBDD = os.listdir('Scripts_cargue_BBDD')
archivos_sql.extend(['Scripts_cargue_BBDD/' + archivo for archivo in archivos_cargue_BBDD])

nombre_zip = 'Scripts_sql.zip'

with ZipFile(nombre_zip, 'w') as zipf:
    for archivo_sql in archivos_sql:
        zipf.write(archivo_sql, os.path.basename(archivo_sql))

print(f"Archivos SQL comprimidos en: {nombre_zip}")

# Descargar el archivo ZIP
files.download('Scripts_sql.zip')

Archivos SQL comprimidos en: Scripts_sql.zip


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Detiene el temporizador de carga
load_end_time = time.time()

In [ ]:
# Calcula el tiempo de carga
load_time = load_end_time - load_start_time
print(f'Tiempo de carga: {load_time:.2f} segundos')

Tiempo de carga: 69.46 segundos


## 5. Interfaz de monitoreo y control en el proceso ETL. <a id="Interfaz"></a>


### 5.1 Métricas de Desempeño

### 5.1.1 Tiempo de Ejecución de la ETL

In [ ]:
# Imprime los tiempos totales
print(f'Tiempo total de extracción: {extraction_time:.2f} segundos')
print(f'Tiempo total de transformación: {transformation_time:.2f} segundos')
print(f'Tiempo total de carga: {load_time:.2f} segundos')

Tiempo total de extracción: 795.82 segundos
Tiempo total de transformación: 244.56 segundos
Tiempo total de carga: 63.24 segundos


### 5.1.2 Utilización de Recursos (CPU, Memoria):

In [ ]:
# Obtén el uso de CPU y memoria
cpu_usage = psutil.cpu_percent()
memory_usage = psutil.virtual_memory().percent

print("Uso de CPU: {}%".format(cpu_usage))
print("Uso de memoria: {}%".format(memory_usage))


Uso de CPU: 47.3%
Uso de memoria: 37.2%
